# IBM Applied Data Science Capstone Course by Coursera

 <h2>Week 5 Final Report</h2>

 <h2>Opening a New Shopping Mall in Kuala Lumpur, Malaysia</h2>

1.Build a dataframe of neighborhoods in Kuala Lumpur, Malaysia by web scraping the data from Wikipedia page
2.Get the geographical coordinates of the neighborhoods
3.Obtain the venue data for the neighborhoods from Foursquare API
4.Explore and cluster the neighborhoods<

 <h2>1. Import libraries</h2>

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.

<h2>2. Scrap data from Wikipedia page into a DataFrame</h2>

In [ ]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_in_Kuala_Lumpur").text

In [ ]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [ ]:
# create a list to store neighborhood data
neighborhoodList = []

In [ ]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [ ]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df.head()

       Neighborhood
0      Alam Damai
1      Ampang, Kuala Lumpur
2      Bandar Menjalara
3      Bandar Sri Permaisuri
4      Bandar Tasik Selatan

In [ ]:
# print the number of rows of the dataframe
kl_df.shape

(70, 1)

<h2>3. Get the geographical coordinates</h2>

In [ ]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Kuala Lumpur, Malaysia'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [ ]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [ ]:
coords

In [ ]:
[[3.0576900000000364, 101.74388000000005],
 [3.158092785454847, 101.74870260727259],
 [3.1903500000000236, 101.62545000000006],
 [3.1039100000000417, 101.71226000000007],
 [3.072620000000029, 101.71471000000008],
 [3.08280000000002, 101.72281000000004],
 [3.1292000000000257, 101.67844000000008],
 [3.1347800000000348, 101.67262000000005],
 [3.111020000000053, 101.66283000000004],
 [3.09898000000004, 101.73499000000004],
 [3.09898000000004, 101.73499000000004],
 [3.12916000000007, 101.68406000000004],
 [3.147770000000037, 101.70855000000006],
 [3.057800000000043, 101.68965000000009],
 [3.1434800000000678, 101.64433000000008],
 [3.1520079194122164, 101.70102829894834],
 [3.129290000000026, 101.69892000000004],
 [3.17381000000006, 101.68276000000009],
 [3.061870000000056, 101.74675000000008],
 [3.163590000000056, 101.69811000000004],
 [3.147980000000075, 101.66798000000006],
 [3.1413156714557724, 101.62607997785557],
 [3.1413156714557724, 101.62607997785557],
 [3.155238081462032, 101.70152004297655],
 [3.08331000000004, 101.70438000000007],
 [3.1363700000000563, 101.68564000000003],
 [3.201630000000023, 101.72107000000005],
 [3.132961658243644, 101.72470311855207],
 [3.179388043398946, 101.67745432110556],
 [3.209500000000048, 101.65874000000008],
 [3.165460000000053, 101.71028000000007],
 [3.166400000000067, 101.73046000000005],
 [3.2175000000000296, 101.63763000000006],
 [3.117110000000025, 101.67349000000007],
 [3.0907400000000393, 101.67733000000004],
 [3.121216228040549, 101.66389741147111],
 [3.147890000000075, 101.69405000000006],
 [3.1592600000000743, 101.69834000000009],
 [3.147890000000075, 101.69405000000006],
 [3.165320000000065, 101.65243000000004],
 [3.0951600000000212, 101.66812000000004],
 [3.133540000000039, 101.71307000000007],
 [3.111867031417114, 101.71326779181086],
 [3.0810200000000236, 101.69724000000008],
 [3.186390000000074, 101.66810000000004],
 [3.1803929473666943, 101.7234139398852],
 [3.175374881541043, 101.69303426939257],
 [3.188160000000039, 101.70415000000008],
 [3.1784341541269487, 101.73701342752202],
 [3.124580000000037, 101.73597000000007],
 [3.162200000000041, 101.65036000000003],
 [3.0726000000000226, 101.68252000000007],
 [3.050640000000044, 101.70613000000009],
 [3.2006600000000276, 101.63337000000007],
 [3.0826300000000515, 101.74671000000006],
 [3.0826900000000705, 101.73689000000007],
 [3.1029700000000275, 101.68471000000005],
 [3.21216000000004, 101.71540000000005],
 [3.069080000000042, 101.74287000000004],
 [3.223570000000052, 101.72399000000007],
 [3.093590000000063, 101.72837000000004],
 [3.2100374598740182, 101.63449832484254],
 [3.1936000000000604, 101.70598000000007],
 [3.1900700000000484, 101.65293000000008],
 [3.0870700000000397, 101.73681000000005],
 [3.1528300000000513, 101.62271000000004],
 [3.1576500000000465, 101.72502000000009],
 [3.22240000000005, 101.67173000000008],
 [3.1806700000000205, 101.70322000000004],
 [3.2039100000000644, 101.73719000000006]]

In [ ]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [ ]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [ ]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df

(70, 3)

In [ ]:
Neighborhood	Latitude	Longitude
0	Alam Damai	3.057690	101.743880
1	Ampang, Kuala Lumpur	3.158093	101.748703
2	Bandar Menjalara	3.190350	101.625450
3	Bandar Sri Permaisuri	3.103910	101.712260
4	Bandar Tasik Selatan	3.072620	101.714710
5	Bandar Tun Razak	3.082800	101.722810
6	Bangsar	3.129200	101.678440
7	Bangsar Park	3.134780	101.672620
8	Bangsar South	3.111020	101.662830
9	Batu 11 Cheras	3.098980	101.734990
10	Batu, Kuala Lumpur	3.098980	101.734990
11	Brickfields	3.129160	101.684060
12	Bukit Bintang	3.147770	101.708550
13	Bukit Jalil	3.057800	101.689650
14	Bukit Kiara	3.143480	101.644330
15	Bukit Nanas	3.152008	101.701028
16	Bukit Petaling	3.129290	101.698920
17	Bukit Tunku	3.173810	101.682760
18	Cheras, Kuala Lumpur	3.061870	101.746750
19	Chow Kit	3.163590	101.698110
20	Damansara Heights	3.147980	101.667980
21	Damansara Town Centre	3.141316	101.626080
22	Damansara, Kuala Lumpur	3.141316	101.626080
23	Dang Wangi	3.155238	101.701520
24	Desa Petaling	3.083310	101.704380
25	Federal Hill, Kuala Lumpur	3.136370	101.685640
26	Happy Garden	3.201630	101.721070
27	Jalan Cochrane, Kuala Lumpur	3.132962	101.724703
28	Jalan Duta	3.179388	101.677454
29	Jinjang	3.209500	101.658740
30	Kampung Baru, Kuala Lumpur	3.165460	101.710280
31	Kampung Datuk Keramat	3.166400	101.730460
32	Kepong	3.217500	101.637630
33	KL Eco City	3.117110	101.673490
34	Kuchai Lama	3.090740	101.677330
35	Lembah Pantai	3.121216	101.663897
36	Maluri	3.147890	101.694050
37	Medan Tuanku	3.159260	101.698340
38	Miharja	3.147890	101.694050
39	Mont Kiara	3.165320	101.652430
40	Pantai Dalam	3.095160	101.668120
41	Pudu, Kuala Lumpur	3.133540	101.713070
42	Putrajaya	3.111867	101.713268
43	Salak South	3.081020	101.697240
44	Segambut	3.186390	101.668100
45	Semarak	3.180393	101.723414
46	Sentul Raya	3.175375	101.693034
47	Setapak	3.188160	101.704150
48	Setiawangsa	3.178434	101.737013
49	Shamelin	3.124580	101.735970
50	Sri Hartamas	3.162200	101.650360
51	Sri Petaling	3.072600	101.682520
52	Sungai Besi	3.050640	101.706130
53	Taman Bukit Maluri	3.200660	101.633370
54	Taman Cheras Hartamas	3.082630	101.746710
55	Taman Connaught	3.082690	101.736890
56	Taman Desa	3.102970	101.684710
57	Taman Ibukota	3.212160	101.715400
58	Taman Len Seng	3.069080	101.742870
59	Taman Melati	3.223570	101.723990
60	Taman Midah	3.093590	101.728370
61	Taman OUG	3.210037	101.634498
62	Taman P. Ramlee	3.193600	101.705980
63	Taman Sri Sinar	3.190070	101.652930
64	Taman Taynton View	3.087070	101.736810
65	Taman Tun Dr Ismail	3.152830	101.622710
66	Taman U-Thant	3.157650	101.725020
67	Taman Wahyu	3.222400	101.671730
68	Titiwangsa	3.180670	101.703220
69	Wangsa Maju	3.203910	101.737190

In [ ]:
# save the DataFrame as CSV file
kl_df.to_csv("kl_df.csv", index=False)

<h2>4. Create a map of Kuala Lumpur with neighborhoods superimposed on top</h2>

In [ ]:
# get the coordinates of Kuala Lumpur
address = 'Kuala Lumpur, Malaysia'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kuala Lumpur, Malaysiae {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kuala Lumpur, Malaysiae 3.1516636, 101.6943028.

In [ ]:
# create map of Toronto using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [ ]:
# save the map as HTML file
map_kl.save('map_kl.html')

<h2>5. Use the Foursquare API to explore the neighborhoods</h2>

In [ ]:
# define Foursquare Credentials and Version
CLIENT_ID = 'your Foursquare ID' # your Foursquare ID
CLIENT_SECRET = 'your Foursquare Secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
Your credentails:
CLIENT_ID: your Foursquare ID
CLIENT_SECRET:your Foursquare Secret

<h2>Now, let's get the top 100 venues that are within a radius of 2000 meters.</h2>

In [ ]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [ ]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(6970, 7)

In [ ]:
Neighborhood	Latitude	Longitude	VenueName	VenueLatitude	VenueLongitude	VenueCategory
0	Alam Damai	3.05769	101.74388	Machi Noodle 妈子面	3.057695	101.746635	Noodle House
1	Alam Damai	3.05769	101.74388	Pengedar Shaklee KL | COD Vivix Shaklee Kuala ...	3.061235	101.740696	Supplement Shop
2	Alam Damai	3.05769	101.74388	628火焰鑫茶室	3.058442	101.747947	Chinese Restaurant
3	Alam Damai	3.05769	101.74388	Ivy Sekinchan Seafood Noodle House 适耕莊特制魚丸海鲜面	3.065749	101.748718	Noodle House
4	Alam Damai	3.05769	101.74388	Restoran Ikbal	3.061134	101.750220	Restaurant

<h2>Let's check how many venues were returned for each neighorhood</h2>

In [ ]:
venues_df.groupby(["Neighborhood"]).count()

In [ ]:
Latitude	Longitude	VenueName	VenueLatitude	VenueLongitude	VenueCategory
Neighborhood						
Alam Damai	100	100	100	100	100	100
Ampang, Kuala Lumpur	100	100	100	100	100	100
Bandar Menjalara	100	100	100	100	100	100
Bandar Sri Permaisuri	100	100	100	100	100	100
Bandar Tasik Selatan	100	100	100	100	100	100
Bandar Tun Razak	100	100	100	100	100	100
Bangsar	100	100	100	100	100	100
Bangsar Park	100	100	100	100	100	100
Bangsar South	100	100	100	100	100	100
Batu 11 Cheras	100	100	100	100	100	100
Batu, Kuala Lumpur	100	100	100	100	100	100
Brickfields	100	100	100	100	100	100
Bukit Bintang	100	100	100	100	100	100
Bukit Jalil	100	100	100	100	100	100
Bukit Kiara	100	100	100	100	100	100
Bukit Nanas	100	100	100	100	100	100
Bukit Petaling	100	100	100	100	100	100
Bukit Tunku	100	100	100	100	100	100
Cheras, Kuala Lumpur	100	100	100	100	100	100
Chow Kit	100	100	100	100	100	100
Damansara Heights	100	100	100	100	100	100
Damansara Town Centre	100	100	100	100	100	100
Damansara, Kuala Lumpur	100	100	100	100	100	100
Dang Wangi	100	100	100	100	100	100
Desa Petaling	100	100	100	100	100	100
Federal Hill, Kuala Lumpur	100	100	100	100	100	100
Happy Garden	100	100	100	100	100	100
Jalan Cochrane, Kuala Lumpur	100	100	100	100	100	100
Jalan Duta	100	100	100	100	100	100
Jinjang	100	100	100	100	100	100
KL Eco City	100	100	100	100	100	100
Kampung Baru, Kuala Lumpur	100	100	100	100	100	100
Kampung Datuk Keramat	100	100	100	100	100	100
Kepong	100	100	100	100	100	100
Kuchai Lama	100	100	100	100	100	100
Lembah Pantai	100	100	100	100	100	100
Maluri	100	100	100	100	100	100
Medan Tuanku	100	100	100	100	100	100
Miharja	100	100	100	100	100	100
Mont Kiara	100	100	100	100	100	100
Pantai Dalam	100	100	100	100	100	100
Pudu, Kuala Lumpur	100	100	100	100	100	100
Putrajaya	78	78	78	78	78	78
Salak South	100	100	100	100	100	100
Segambut	100	100	100	100	100	100
Semarak	100	100	100	100	100	100
Sentul Raya	100	100	100	100	100	100
Setapak	100	100	100	100	100	100
Setiawangsa	100	100	100	100	100	100
Shamelin	100	100	100	100	100	100
Sri Hartamas	100	100	100	100	100	100
Sri Petaling	100	100	100	100	100	100
Sungai Besi	100	100	100	100	100	100
Taman Bukit Maluri	100	100	100	100	100	100
Taman Cheras Hartamas	100	100	100	100	100	100
Taman Connaught	100	100	100	100	100	100
Taman Desa	100	100	100	100	100	100
Taman Ibukota	100	100	100	100	100	100
Taman Len Seng	99	99	99	99	99	99
Taman Melati	100	100	100	100	100	100
Taman Midah	100	100	100	100	100	100
Taman OUG	100	100	100	100	100	100
Taman P. Ramlee	93	93	93	93	93	93
Taman Sri Sinar	100	100	100	100	100	100
Taman Taynton View	100	100	100	100	100	100
Taman Tun Dr Ismail	100	100	100	100	100	100
Taman U-Thant	100	100	100	100	100	100
Taman Wahyu	100	100	100	100	100	100
Titiwangsa	100	100	100	100	100	100
Wangsa Maju	100	100	100	100	100	100

<h2>Let's find out how many unique categories can be curated from all the returned venues</h2>

In [ ]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 304 uniques categories.

In [ ]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

In [ ]:
array(['Noodle House', 'Supplement Shop', 'Chinese Restaurant',
       'Restaurant', 'Food Court', 'Vegetarian / Vegan Restaurant',
       'Asian Restaurant', 'Dim Sum Restaurant', 'Snack Place',
       'Other Great Outdoors', 'Seafood Restaurant', 'Spa', 'Food Truck',
       'Café', 'Park', 'Chinese Breakfast Place', 'Indian Restaurant',
       'Japanese Restaurant', 'Outlet Store', 'Convenience Store',
       'Bubble Tea Shop', 'Dessert Shop', 'Farmers Market',
       'Cantonese Restaurant', 'Malay Restaurant', 'Bakery',
       'Hakka Restaurant', 'Supermarket', 'Steakhouse', 'Pet Store',
       'Middle Eastern Restaurant', 'Badminton Court',
       'Athletics & Sports', 'Hookah Bar', 'Winery', 'Burger Joint',
       'Gym / Fitness Center', 'Bistro', 'Grocery Store',
       'Halal Restaurant', 'College Bookstore', 'Flea Market',
       'Vietnamese Restaurant', 'Italian Restaurant', 'Coffee Shop',
       'Juice Bar', 'Korean Restaurant', 'Pizza Place', 'Ice Cream Shop',
       'Massage Studio'], dtype=object)

In [ ]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

In [ ]:
True

<h2>6. Analyze Each Neighborhood</h2>

In [ ]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(6970, 305)

In [ ]:
Neighborhoods	Accessories Store	Adult Boutique	Afghan Restaurant	African Restaurant	Airport	Airport Service	Airport Terminal	American Restaurant	Aquarium	Arcade	Art Gallery	Art Museum	Arts & Crafts Store	Asian Restaurant	Athletics & Sports	Auditorium	Auto Dealership	Auto Garage	Auto Workshop	Automotive Shop	BBQ Joint	Badminton Court	Bagel Shop	Bakery	Bar	Baseball Field	Basketball Court	Beach	Bed & Breakfast	Beer Bar	Beer Garden	Belgian Restaurant	Bistro	Board Shop	Boat or Ferry	Bookstore	Botanical Garden	Boutique	Bowling Alley	Breakfast Spot	Brewery	Bridal Shop	Bubble Tea Shop	Buffet	Building	Burger Joint	Bus Station	Business Service	Butcher	Cafeteria	Café	Campground	Candy Store	Cantonese Restaurant	Casino	Chettinad Restaurant	Chinese Breakfast Place	Chinese Restaurant	Circus	Clothing Store	Club House	Cocktail Bar	Coffee Shop	College Bookstore	Comedy Club	Comfort Food Restaurant	Concert Hall	Convenience Store	Convention Center	Cosmetics Shop	Cupcake Shop	Cycle Studio	Dance Studio	Deli / Bodega	Department Store	Design Studio	Dessert Shop	Dim Sum Restaurant	Diner	Dive Bar	Dive Shop	Dog Run	Donut Shop	Dumpling Restaurant	Eastern European Restaurant	Electronics Store	Event Space	Exhibit	Falafel Restaurant	Farm	Farmers Market	Fast Food Restaurant	Field	Fish & Chips Shop	Fish Market	Fishing Store	Flea Market	Flower Shop	Food	Food & Drink Shop	Food Court	Food Stand	Food Truck	Fountain	Frame Store	French Restaurant	Fried Chicken Joint	Frozen Yogurt Shop	Fruit & Vegetable Store	Furniture / Home Store	Gaming Cafe	Garden	Garden Center	Gas Station	Gastropub	Gay Bar	General Entertainment	German Restaurant	Gift Shop	Golf Course	Gourmet Shop	Government Building	Grilled Meat Restaurant	Grocery Store	Gym	Gym / Fitness Center	Gym Pool	Hainan Restaurant	Hakka Restaurant	Halal Restaurant	Harbor / Marina	Hawaiian Restaurant	Health & Beauty Service	High School	Himalayan Restaurant	Historic Site	History Museum	Hockey Arena	Hong Kong Restaurant	Hookah Bar	Hostel	Hot Dog Joint	Hot Spring	Hotel	Hotel Bar	Hotel Pool	Hotpot Restaurant	Housing Development	Hunan Restaurant	IT Services	Ice Cream Shop	Indian Restaurant	Indonesian Restaurant	Italian Restaurant	Japanese Curry Restaurant	Japanese Restaurant	Jazz Club	Jewelry Store	Juice Bar	Karaoke Bar	Kebab Restaurant	Kids Store	Korean Restaurant	Kushikatsu Restaurant	Lake	Latin American Restaurant	Leather Goods Store	Light Rail Station	Lingerie Store	Lounge	Malay Restaurant	Market	Martial Arts Dojo	Massage Studio	Medical Center	Mediterranean Restaurant	Men's Store	Mexican Restaurant	Middle Eastern Restaurant	Miscellaneous Shop	Mobile Phone Shop	Modern European Restaurant	Monument / Landmark	Moroccan Restaurant	Mosque	Motel	Motorcycle Shop	Mountain	Movie Theater	Multiplex	Museum	Music Store	Music Venue	Nail Salon	Neighborhood	Night Market	Nightclub	Noodle House	North Indian Restaurant	Office	Optical Shop	Other Great Outdoors	Other Nightlife	Outlet Store	Pakistani Restaurant	Palace	Paper / Office Supplies Store	Park	Performing Arts Venue	Pet Store	Pharmacy	Piano Bar	Pie Shop	Pizza Place	Playground	Plaza	Poke Place	Pool	Pool Hall	Pub	Public Art	Racetrack	Ramen Restaurant	Record Shop	Residential Building (Apartment / Condo)	Resort	Rest Area	Restaurant	Road	Rock Climbing Spot	Rock Club	Sake Bar	Salad Place	Salon / Barbershop	Sandwich Place	Satay Restaurant	Scandinavian Restaurant	Scenic Lookout	Science Museum	Seafood Restaurant	Shabu-Shabu Restaurant	Shoe Store	Shop & Service	Shopping Mall	Skate Park	Ski Area	Ski Lodge	Smoke Shop	Smoothie Shop	Snack Place	Soccer Field	Soccer Stadium	Soup Place	South American Restaurant	South Indian Restaurant	Spa	Spanish Restaurant	Speakeasy	Spiritual Center	Sporting Goods Shop	Sports Bar	Sports Club	Stadium	Stationery Store	Steakhouse	Street Food Gathering	Supermarket	Supplement Shop	Surf Spot	Sushi Restaurant	Taiwanese Restaurant	Tapas Restaurant	Tattoo Parlor	Tea Room	Temple	Tennis Court	Tex-Mex Restaurant	Thai Restaurant	Theater	Theme Restaurant	Thrift / Vintage Store	Toll Plaza	Tourist Information Center	Toy / Game Store	Track	Track Stadium	Trail	Train Station	Tunnel	Turkish Restaurant	Udon Restaurant	Vape Store	Vegetarian / Vegan Restaurant	Video Game Store	Vietnamese Restaurant	Volleyball Court	Warehouse Store	Whisky Bar	Wine Bar	Winery	Wings Joint	Women's Store	Yoga Studio	Zoo
0	Alam Damai	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0
1	Alam Damai	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0
2	Alam Damai	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0
3	Alam Damai	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0
4	Alam Damai	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0

<h2>Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category</h2>

In [ ]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(70, 305)

Neighborhoods	Accessories Store	Adult Boutique	Afghan Restaurant	African Restaurant	Airport	Airport Service	Airport Terminal	American Restaurant	Aquarium	Arcade	Art Gallery	Art Museum	Arts & Crafts Store	Asian Restaurant	Athletics & Sports	Auditorium	Auto Dealership	Auto Garage	Auto Workshop	Automotive Shop	BBQ Joint	Badminton Court	Bagel Shop	Bakery	Bar	Baseball Field	Basketball Court	Beach	Bed & Breakfast	Beer Bar	Beer Garden	Belgian Restaurant	Bistro	Board Shop	Boat or Ferry	Bookstore	Botanical Garden	Boutique	Bowling Alley	Breakfast Spot	Brewery	Bridal Shop	Bubble Tea Shop	Buffet	Building	Burger Joint	Bus Station	Business Service	Butcher	Cafeteria	Café	Campground	Candy Store	Cantonese Restaurant	Casino	Chettinad Restaurant	Chinese Breakfast Place	Chinese Restaurant	Circus	Clothing Store	Club House	Cocktail Bar	Coffee Shop	College Bookstore	Comedy Club	Comfort Food Restaurant	Concert Hall	Convenience Store	Convention Center	Cosmetics Shop	Cupcake Shop	Cycle Studio	Dance Studio	Deli / Bodega	Department Store	Design Studio	Dessert Shop	Dim Sum Restaurant	Diner	Dive Bar	Dive Shop	Dog Run	Donut Shop	Dumpling Restaurant	Eastern European Restaurant	Electronics Store	Event Space	Exhibit	Falafel Restaurant	Farm	Farmers Market	Fast Food Restaurant	Field	Fish & Chips Shop	Fish Market	Fishing Store	Flea Market	Flower Shop	Food	Food & Drink Shop	Food Court	Food Stand	Food Truck	Fountain	Frame Store	French Restaurant	Fried Chicken Joint	Frozen Yogurt Shop	Fruit & Vegetable Store	Furniture / Home Store	Gaming Cafe	Garden	Garden Center	Gas Station	Gastropub	Gay Bar	General Entertainment	German Restaurant	Gift Shop	Golf Course	Gourmet Shop	Government Building	Grilled Meat Restaurant	Grocery Store	Gym	Gym / Fitness Center	Gym Pool	Hainan Restaurant	Hakka Restaurant	Halal Restaurant	Harbor / Marina	Hawaiian Restaurant	Health & Beauty Service	High School	Himalayan Restaurant	Historic Site	History Museum	Hockey Arena	Hong Kong Restaurant	Hookah Bar	Hostel	Hot Dog Joint	Hot Spring	Hotel	Hotel Bar	Hotel Pool	Hotpot Restaurant	Housing Development	Hunan Restaurant	IT Services	Ice Cream Shop	Indian Restaurant	Indonesian Restaurant	Italian Restaurant	Japanese Curry Restaurant	Japanese Restaurant	Jazz Club	Jewelry Store	Juice Bar	Karaoke Bar	Kebab Restaurant	Kids Store	Korean Restaurant	Kushikatsu Restaurant	Lake	Latin American Restaurant	Leather Goods Store	Light Rail Station	Lingerie Store	Lounge	Malay Restaurant	Market	Martial Arts Dojo	Massage Studio	Medical Center	Mediterranean Restaurant	Men's Store	Mexican Restaurant	Middle Eastern Restaurant	Miscellaneous Shop	Mobile Phone Shop	Modern European Restaurant	Monument / Landmark	Moroccan Restaurant	Mosque	Motel	Motorcycle Shop	Mountain	Movie Theater	Multiplex	Museum	Music Store	Music Venue	Nail Salon	Neighborhood	Night Market	Nightclub	Noodle House	North Indian Restaurant	Office	Optical Shop	Other Great Outdoors	Other Nightlife	Outlet Store	Pakistani Restaurant	Palace	Paper / Office Supplies Store	Park	Performing Arts Venue	Pet Store	Pharmacy	Piano Bar	Pie Shop	Pizza Place	Playground	Plaza	Poke Place	Pool	Pool Hall	Pub	Public Art	Racetrack	Ramen Restaurant	Record Shop	Residential Building (Apartment / Condo)	Resort	Rest Area	Restaurant	Road	Rock Climbing Spot	Rock Club	Sake Bar	Salad Place	Salon / Barbershop	Sandwich Place	Satay Restaurant	Scandinavian Restaurant	Scenic Lookout	Science Museum	Seafood Restaurant	Shabu-Shabu Restaurant	Shoe Store	Shop & Service	Shopping Mall	Skate Park	Ski Area	Ski Lodge	Smoke Shop	Smoothie Shop	Snack Place	Soccer Field	Soccer Stadium	Soup Place	South American Restaurant	South Indian Restaurant	Spa	Spanish Restaurant	Speakeasy	Spiritual Center	Sporting Goods Shop	Sports Bar	Sports Club	Stadium	Stationery Store	Steakhouse	Street Food Gathering	Supermarket	Supplement Shop	Surf Spot	Sushi Restaurant	Taiwanese Restaurant	Tapas Restaurant	Tattoo Parlor	Tea Room	Temple	Tennis Court	Tex-Mex Restaurant	Thai Restaurant	Theater	Theme Restaurant	Thrift / Vintage Store	Toll Plaza	Tourist Information Center	Toy / Game Store	Track	Track Stadium	Trail	Train Station	Tunnel	Turkish Restaurant	Udon Restaurant	Vape Store	Vegetarian / Vegan Restaurant	Video Game Store	Vietnamese Restaurant	Volleyball Court	Warehouse Store	Whisky Bar	Wine Bar	Winery	Wings Joint	Women's Store	Yoga Studio	Zoo
0	Alam Damai	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.060000	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.010000	0.00	0.020000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.050000	0.00	0.00	0.020000	0.00	0.00	0.010000	0.160000	0.00	0.00	0.00	0.00	0.000000	0.01	0.00	0.00	0.00	0.080000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.010000	0.030000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.010000	0.000000	0.00	0.00	0.00	0.00	0.010000	0.000000	0.000000	0.000000	0.020000	0.00	0.030000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.01	0.00	0.01	0.00	0.00	0.010000	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.030000	0.000000	0.00	0.00	0.010000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.090000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.060000	0.00	0.00	0.00	0.010000	0.00	0.01	0.00	0.00	0.00	0.010000	0.00	0.020000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.020000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.000000	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.020000	0.00	0.010000	0.010000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.010000	0.000000	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.00
1	Ampang, Kuala Lumpur	0.000000	0.00	0.01	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.010000	0.00	0.00	0.060000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.020000	0.000000	0.00	0.010000	0.00	0.000000	0.00	0.00	0.00	0.010000	0.00	0.00	0.01	0.00	0.000000	0.00	0.010000	0.00	0.00	0.000000	0.00	0.00	0.020000	0.000000	0.000000	0.00	0.00	0.020000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.060000	0.00	0.00	0.00	0.00	0.070000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.020000	0.000000	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.000000	0.010000	0.00	0.00	0.00	0.00	0.020000	0.000000	0.000000	0.010000	0.000000	0.00	0.040000	0.00	0.00	0.00	0.00	0.000000	0.00	0.01	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.02	0.00	0.00	0.00	0.00	0.000000	0.010000	0.00	0.00	0.010000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.020000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.00	0.020000	0.010000	0.010000	0.02	0.00	0.000000	0.000000	0.00	0.03	0.000000	0.000000	0.00	0.050000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.150000	0.000000	0.00	0.01	0.00	0.00	0.01	0.01	0.020000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.020000	0.000000	0.00	0.00	0.020000	0.010000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.020000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.010000	0.00	0.00	0.00	0.01	0.00	0.000000	0.000000	0.00	0.030000	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
2	Bandar Menjalara	0.000000	0.00	0.00	0.01	0.000000	0.000000	0.00	0.00	0.00	0.010000	0.000000	0.00	0.00	0.040000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.010000	0.00	0.020000	0.00	0.000000	0.01	0.00	0.00	0.010000	0.00	0.00	0.01	0.00	0.000000	0.00	0.010000	0.00	0.00	0.000000	0.00	0.01	0.000000	0.000000	0.000000	0.00	0.00	0.110000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.100000	0.00	0.00	0.00	0.00	0.060000	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.01	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.010000	0.00	0.00	0.00	0.00	0.000000	0.000000	0.010000	0.030000	0.010000	0.01	0.010000	0.00	0.00	0.00	0.01	0.010000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.02	0.01	0.00	0.00	0.00	0.010000	0.000000	0.00	0.01	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.00	0.00	0.00	0.010000	0.010000	0.000000	0.00	0.00	0.050000	0.000000	0.00	0.01	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.010000	0.00	0.01	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.000000	0.01	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.020000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.040000	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.020000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.030000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.030000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.010000	0.01	0.00	0.00	0.01	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.040000	0.00	0.01	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.030000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
3	Bandar Sri Permaisuri	0.010000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.010000	0.00	0.00	0.060000	0.000000	0.01	0.00	0.020000	0.00	0.010000	0.000000	0.000000	0.00	0.020000	0.000000	0.00	0.000000	0.00	0.010000	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.000000	0.00	0.020000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.010000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.140000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.050000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.010000	0.00	0.00	0.01	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.030000	0.00	0.00	0.00	0.00	0.010000	0.010000	0.020000	0.000000	0.000000	0.00	0.020000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.02	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.010000	0.00	0.000000	0.00	0.00	0.010000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.00	0.010000	0.020000	0.020000	0.00	0.00	0.000000	0.010000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.010000	0.00	0.00	0.02	0.00	0.00	0.090000	0.020000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.01	0.00	0.000000	0.010000	0.000000	0.010000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.030000	0.000000	0.010000	0.00	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.020000	0.000000	0.00	0.010000	0.00	0.00	0.020000	0.010000	0.00	0.00	0.00	0.00	0.040000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.020000	0.010000	0.010000	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.010000	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.010000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.01	0.00	0.00	0.00	0.00	0.00	0.00	0.00
4	Bandar Tasik Selatan	0.010000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.01	0.080000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.010000	0.010000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.01	0.00	0.00	0.010000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.040000	0.000000	0.000000	0.00	0.00	0.020000	0.00	0.00	0.000000	0.01	0.00	0.000000	0.070000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.01	0.00	0.050000	0.00	0.00	0.00	0.00	0.000000	0.00	0.02	0.00	0.020000	0.010000	0.000000	0.00	0.00	0.00	0.01	0.01	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.010000	0.00	0.00	0.00	0.00	0.030000	0.000000	0.000000	0.000000	0.020000	0.00	0.020000	0.00	0.00	0.00	0.01	0.000000	0.00	0.01	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.01	0.01	0.00	0.000000	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.00	0.020000	0.030000	0.020000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.01	0.00	0.00	0.110000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.000000	0.01	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.000000	0.030000	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.01	0.00	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.000000	0.010000	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.000000	0.020000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.020000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.01	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.010000	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.010000	0.00	0.000000	0.000000	0.01	0.000000	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.020000	0.00	0.00	0.01	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.00
5	Bandar Tun Razak	0.010000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.01	0.090000	0.000000	0.01	0.00	0.000000	0.00	0.000000	0.010000	0.000000	0.00	0.010000	0.000000	0.00	0.010000	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.020000	0.00	0.00	0.010000	0.00	0.00	0.020000	0.000000	0.000000	0.00	0.00	0.040000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.190000	0.00	0.00	0.00	0.00	0.030000	0.00	0.00	0.01	0.00	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.010000	0.00	0.00	0.00	0.00	0.020000	0.000000	0.010000	0.000000	0.020000	0.00	0.010000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.000000	0.000000	0.000000	0.02	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.030000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.020000	0.00	0.010000	0.00	0.00	0.01	0.00	0.00	0.060000	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.01	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.000000	0.020000	0.00	0.00	0.00	0.000000	0.01	0.00	0.00	0.01	0.00	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.020000	0.000000	0.000000	0.00	0.010000	0.020000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.020000	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.020000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.010000	0.000000	0.000000	0.00	0.00	0.010000	0.00	0.00	0.00	0.01	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.000000	0.01	0.000000	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.010000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.010000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
6	Bangsar	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.02	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.020000	0.010000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.010000	0.00	0.00	0.01	0.00	0.020000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.050000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.05	0.00	0.00	0.030000	0.00	0.00	0.00	0.00	0.010000	0.00	0.02	0.00	0.00	0.000000	0.00	0.01	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.000000	0.020000	0.000000	0.000000	0.020000	0.00	0.010000	0.00	0.00	0.02	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.02	0.00	0.01	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.01	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.050000	0.02	0.00	0.000000	0.00	0.00	0.00	0.030000	0.090000	0.000000	0.00	0.00	0.010000	0.000000	0.00	0.02	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.010000	0.000000	0.00	0.01	0.01	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.02	0.010000	0.00	0.000000	0.00	0.00	0.010000	0.000000	0.010000	0.000000	0.00	0.00	0.01	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.01	0.000000	0.010000	0.01	0.00	0.010000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.010000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.05	0.00	0.000000	0.000000	0.00	0.01	0.010000	0.010000	0.000000	0.000000	0.00	0.01	0.020000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.01	0.030000	0.00	0.010000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.01	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.010000	0.000000	0.00	0.00	0.01	0.00	0.00	0.00	0.01	0.00
7	Bangsar Park	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.02	0.020000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.010000	0.000000	0.00	0.010000	0.030000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.020000	0.00	0.00	0.00	0.00	0.030000	0.00	0.000000	0.00	0.01	0.000000	0.00	0.00	0.020000	0.000000	0.000000	0.00	0.00	0.070000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.03	0.00	0.00	0.030000	0.00	0.00	0.00	0.00	0.010000	0.00	0.02	0.00	0.00	0.000000	0.00	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.000000	0.010000	0.000000	0.000000	0.000000	0.00	0.010000	0.00	0.00	0.01	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.050000	0.02	0.00	0.000000	0.00	0.00	0.00	0.040000	0.090000	0.000000	0.01	0.00	0.000000	0.000000	0.00	0.01	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.020000	0.000000	0.00	0.01	0.01	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.000000	0.010000	0.000000	0.00	0.00	0.01	0.000000	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.000000	0.010000	0.01	0.00	0.010000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.000000	0.010000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.04	0.00	0.000000	0.000000	0.00	0.01	0.000000	0.010000	0.000000	0.000000	0.00	0.01	0.020000	0.01	0.00	0.00	0.00	0.01	0.000000	0.000000	0.01	0.040000	0.00	0.020000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.030000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.02	0.00	0.00	0.00	0.01	0.00
8	Bangsar South	0.010000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.060000	0.000000	0.00	0.010000	0.00	0.000000	0.00	0.00	0.01	0.000000	0.00	0.00	0.01	0.00	0.010000	0.00	0.020000	0.00	0.00	0.010000	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.050000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.030000	0.00	0.02	0.00	0.00	0.030000	0.00	0.00	0.00	0.00	0.020000	0.00	0.02	0.00	0.00	0.000000	0.01	0.03	0.00	0.010000	0.000000	0.000000	0.00	0.01	0.00	0.01	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.000000	0.010000	0.000000	0.010000	0.010000	0.00	0.000000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.01	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.030000	0.00	0.00	0.000000	0.00	0.00	0.00	0.050000	0.010000	0.000000	0.00	0.00	0.060000	0.000000	0.02	0.02	0.000000	0.000000	0.00	0.000000	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.040000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.02	0.010000	0.00	0.000000	0.00	0.01	0.000000	0.000000	0.000000	0.010000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.010000	0.010000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.000000	0.020000	0.000000	0.00	0.000000	0.00	0.01	0.010000	0.010000	0.00	0.00	0.00	0.00	0.010000	0.01	0.01	0.00	0.03	0.00	0.000000	0.010000	0.00	0.00	0.010000	0.010000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.00	0.01	0.00	0.000000	0.000000	0.00	0.010000	0.00	0.010000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.000000	0.01	0.00	0.00	0.000000	0.00	0.01	0.00	0.000000	0.010000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
9	Batu 11 Cheras	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.060000	0.000000	0.01	0.00	0.000000	0.00	0.000000	0.010000	0.000000	0.00	0.010000	0.010000	0.00	0.010000	0.00	0.000000	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.000000	0.00	0.020000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.030000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.240000	0.00	0.00	0.00	0.00	0.030000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.010000	0.000000	0.010000	0.000000	0.010000	0.00	0.030000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.000000	0.010000	0.00	0.00	0.010000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.030000	0.00	0.00	0.00	0.020000	0.010000	0.000000	0.00	0.00	0.010000	0.000000	0.00	0.01	0.000000	0.000000	0.00	0.010000	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.050000	0.010000	0.00	0.01	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.01	0.00	0.000000	0.000000	0.000000	0.030000	0.00	0.00	0.00	0.000000	0.00	0.01	0.00	0.00	0.00	0.000000	0.00	0.010000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.020000	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.000000	0.00	0.00	0.00	0.00	0.040000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.000000	0.00	0.00	0.040000	0.00	0.00	0.00	0.01	0.00	0.000000	0.000000	0.00	0.020000	0.00	0.000000	0.000000	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.030000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
10	Batu, Kuala Lumpur	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.060000	0.000000	0.01	0.00	0.000000	0.00	0.000000	0.010000	0.000000	0.00	0.010000	0.010000	0.00	0.010000	0.00	0.000000	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.000000	0.00	0.020000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.030000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.240000	0.00	0.00	0.00	0.00	0.030000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.010000	0.000000	0.010000	0.000000	0.010000	0.00	0.030000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.000000	0.010000	0.00	0.00	0.010000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.030000	0.00	0.00	0.00	0.020000	0.010000	0.000000	0.00	0.00	0.010000	0.000000	0.00	0.01	0.000000	0.000000	0.00	0.010000	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.050000	0.010000	0.00	0.01	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.01	0.00	0.000000	0.000000	0.000000	0.030000	0.00	0.00	0.00	0.000000	0.00	0.01	0.00	0.00	0.00	0.000000	0.00	0.010000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.020000	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.000000	0.00	0.00	0.00	0.00	0.040000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.000000	0.00	0.00	0.040000	0.00	0.00	0.00	0.01	0.00	0.000000	0.000000	0.00	0.020000	0.00	0.000000	0.000000	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.030000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
11	Brickfields	0.000000	0.00	0.00	0.01	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.01	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.020000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.01	0.00	0.010000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.050000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.020000	0.00	0.04	0.00	0.00	0.020000	0.00	0.00	0.00	0.00	0.010000	0.00	0.01	0.00	0.00	0.000000	0.00	0.01	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.010000	0.00	0.00	0.00	0.00	0.000000	0.020000	0.000000	0.000000	0.020000	0.01	0.000000	0.00	0.00	0.02	0.00	0.000000	0.00	0.00	0.000000	0.020000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.02	0.01	0.01	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.01	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.060000	0.02	0.00	0.000000	0.00	0.00	0.00	0.040000	0.070000	0.000000	0.01	0.00	0.020000	0.000000	0.00	0.01	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.01	0.00	0.01	0.020000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.02	0.010000	0.01	0.010000	0.00	0.00	0.010000	0.000000	0.000000	0.000000	0.00	0.00	0.01	0.000000	0.00	0.00	0.00	0.00	0.00	0.020000	0.01	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.010000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.05	0.00	0.000000	0.000000	0.00	0.01	0.030000	0.000000	0.000000	0.000000	0.00	0.02	0.020000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.01	0.040000	0.00	0.000000	0.000000	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.01	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
12	Bukit Bintang	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.01	0.010000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.010000	0.000000	0.00	0.010000	0.020000	0.00	0.000000	0.00	0.000000	0.01	0.00	0.00	0.000000	0.00	0.00	0.01	0.00	0.030000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.050000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.020000	0.00	0.05	0.00	0.01	0.070000	0.00	0.00	0.00	0.00	0.020000	0.00	0.02	0.00	0.00	0.000000	0.00	0.01	0.00	0.010000	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.000000	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.01	0.00	0.000000	0.100000	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.02	0.01	0.040000	0.000000	0.01	0.01	0.000000	0.000000	0.00	0.010000	0.00	0.000000	0.01	0.01	0.00	0.01	0.05	0.000000	0.000000	0.00	0.01	0.00	0.01	0.00	0.00	0.020000	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.000000	0.00	0.00	0.010000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.010000	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.01	0.00	0.00	0.01	0.000000	0.020000	0.000000	0.00	0.000000	0.00	0.01	0.000000	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.01	0.00	0.03	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.000000	0.00	0.00	0.010000	0.01	0.00	0.00	0.01	0.00	0.000000	0.000000	0.00	0.000000	0.01	0.000000	0.000000	0.00	0.010000	0.00	0.01	0.00	0.02	0.00	0.00	0.00	0.020000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.01	0.000000	0.010000	0.00	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
13	Bukit Jalil	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.030000	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.010000	0.010000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.000000	0.01	0.010000	0.00	0.00	0.010000	0.00	0.00	0.010000	0.000000	0.010000	0.00	0.00	0.140000	0.00	0.00	0.010000	0.00	0.00	0.000000	0.080000	0.00	0.00	0.00	0.00	0.080000	0.00	0.00	0.00	0.01	0.020000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.000000	0.000000	0.01	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.030000	0.000000	0.00	0.010000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.030000	0.00	0.02	0.00	0.01	0.01	0.01	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.01	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.01	0.00	0.00	0.020000	0.020000	0.000000	0.00	0.00	0.030000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.040000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.010000	0.000000	0.01	0.02	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.010000	0.00	0.01	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.010000	0.010000	0.00	0.00	0.000000	0.000000	0.010000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.020000	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.010000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.01	0.000000	0.00	0.000000	0.000000	0.00	0.010000	0.01	0.00	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.020000	0.00	0.010000	0.000000	0.00	0.00	0.01	0.00	0.00	0.00	0.00	0.00
14	Bukit Kiara	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.030000	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.010000	0.010000	0.00	0.040000	0.030000	0.00	0.010000	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.01	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.050000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.040000	0.00	0.00	0.00	0.01	0.030000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.01	0.000000	0.01	0.00	0.00	0.000000	0.000000	0.000000	0.01	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.010000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.01	0.00	0.00	0.01	0.00	0.020000	0.00	0.00	0.00	0.01	0.01	0.00	0.00	0.00	0.000000	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.00	0.000000	0.00	0.00	0.00	0.020000	0.030000	0.000000	0.02	0.00	0.020000	0.000000	0.00	0.01	0.000000	0.000000	0.00	0.000000	0.00	0.010000	0.00	0.00	0.00	0.00	0.02	0.050000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.01	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.01	0.00	0.000000	0.000000	0.000000	0.010000	0.01	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.000000	0.02	0.000000	0.000000	0.00	0.01	0.010000	0.000000	0.000000	0.00	0.000000	0.000000	0.04	0.01	0.00	0.00	0.00	0.00	0.00	0.000000	0.030000	0.000000	0.00	0.000000	0.01	0.01	0.000000	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.03	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.01	0.000000	0.00	0.01	0.00	0.00	0.01	0.000000	0.000000	0.00	0.010000	0.00	0.010000	0.000000	0.00	0.020000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.030000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.010000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.010000	0.000000	0.00	0.00	0.03	0.00	0.00	0.01	0.01	0.01
15	Bukit Nanas	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.010000	0.000000	0.00	0.000000	0.020000	0.00	0.000000	0.00	0.000000	0.01	0.00	0.00	0.010000	0.00	0.00	0.01	0.00	0.010000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.01	0.000000	0.000000	0.000000	0.00	0.00	0.040000	0.00	0.00	0.000000	0.00	0.01	0.000000	0.030000	0.00	0.00	0.00	0.02	0.020000	0.00	0.00	0.00	0.00	0.030000	0.00	0.01	0.00	0.00	0.000000	0.00	0.01	0.00	0.010000	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.000000	0.010000	0.000000	0.000000	0.000000	0.00	0.020000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.01	0.000000	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.02	0.00	0.000000	0.130000	0.00	0.01	0.000000	0.00	0.00	0.01	0.000000	0.010000	0.010000	0.02	0.00	0.010000	0.000000	0.01	0.01	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.01	0.00	0.00	0.00	0.04	0.010000	0.000000	0.00	0.01	0.00	0.00	0.01	0.00	0.010000	0.00	0.00	0.00	0.02	0.01	0.00	0.00	0.000000	0.00	0.00	0.020000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.020000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.000000	0.010000	0.000000	0.00	0.000000	0.00	0.01	0.000000	0.000000	0.00	0.00	0.01	0.00	0.010000	0.00	0.00	0.00	0.03	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.020000	0.00	0.01	0.020000	0.01	0.01	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.01	0.000000	0.000000	0.00	0.010000	0.00	0.02	0.00	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.00	0.000000	0.01	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.01	0.000000	0.010000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
16	Bukit Petaling	0.010000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.020000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.01	0.00	0.010000	0.00	0.010000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.080000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.110000	0.00	0.02	0.00	0.01	0.030000	0.00	0.00	0.00	0.00	0.020000	0.00	0.01	0.00	0.00	0.000000	0.00	0.00	0.00	0.020000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.000000	0.010000	0.000000	0.000000	0.010000	0.01	0.020000	0.00	0.01	0.01	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.01	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.02	0.00	0.000000	0.080000	0.02	0.00	0.000000	0.00	0.00	0.00	0.000000	0.040000	0.000000	0.00	0.00	0.010000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.050000	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.02	0.000000	0.00	0.00	0.010000	0.01	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.020000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.010000	0.01	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.020000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.01	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.010000	0.00	0.02	0.020000	0.00	0.02	0.00	0.00	0.00	0.000000	0.000000	0.01	0.010000	0.00	0.000000	0.000000	0.00	0.010000	0.00	0.00	0.00	0.01	0.00	0.01	0.00	0.000000	0.01	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.020000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
17	Bukit Tunku	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.010000	0.020000	0.00	0.00	0.030000	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.030000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.01	0.00	0.020000	0.00	0.00	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.080000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.110000	0.00	0.03	0.00	0.00	0.030000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.010000	0.010000	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.020000	0.000000	0.000000	0.000000	0.010000	0.01	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.01	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.020000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.01	0.00	0.010000	0.01	0.00	0.000000	0.040000	0.00	0.00	0.000000	0.00	0.00	0.00	0.020000	0.040000	0.000000	0.00	0.00	0.040000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.010000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.020000	0.00	0.01	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.01	0.00	0.00	0.020000	0.01	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.020000	0.000000	0.01	0.00	0.00	0.00	0.00	0.01	0.00	0.000000	0.030000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.00	0.02	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.040000	0.00	0.00	0.00	0.00	0.00	0.000000	0.010000	0.00	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.010000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.01	0.00	0.00	0.00	0.00	0.00	0.00	0.00
18	Cheras, Kuala Lumpur	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.060000	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.010000	0.00	0.010000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.030000	0.00	0.00	0.020000	0.00	0.00	0.010000	0.130000	0.00	0.00	0.00	0.00	0.010000	0.01	0.00	0.00	0.00	0.060000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.010000	0.020000	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.010000	0.000000	0.00	0.00	0.00	0.00	0.010000	0.000000	0.000000	0.000000	0.020000	0.00	0.010000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.01	0.01	0.02	0.00	0.00	0.010000	0.020000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.00	0.00	0.000000	0.010000	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.030000	0.000000	0.00	0.00	0.010000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.070000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.010000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.060000	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.010000	0.010000	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.000000	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.010000	0.030000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.030000	0.000000	0.000000	0.000000	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.020000	0.00	0.010000	0.010000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.010000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.00
19	Chow Kit	0.000000	0.01	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.030000	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.010000	0.000000	0.00	0.000000	0.020000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.01	0.00	0.010000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.01	0.000000	0.000000	0.000000	0.00	0.00	0.010000	0.00	0.00	0.000000	0.00	0.01	0.000000	0.050000	0.00	0.02	0.00	0.02	0.030000	0.00	0.00	0.00	0.01	0.010000	0.00	0.01	0.00	0.00	0.010000	0.00	0.00	0.00	0.000000	0.020000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.010000	0.000000	0.000000	0.000000	0.010000	0.00	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.110000	0.00	0.01	0.000000	0.00	0.00	0.01	0.000000	0.010000	0.010000	0.01	0.00	0.010000	0.000000	0.01	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.01	0.00	0.00	0.00	0.01	0.090000	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.010000	0.00	0.01	0.00	0.01	0.01	0.00	0.00	0.000000	0.00	0.00	0.010000	0.02	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.000000	0.010000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.020000	0.00	0.00	0.01	0.00	0.020000	0.00	0.01	0.00	0.03	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.020000	0.00	0.00	0.010000	0.00	0.00	0.00	0.01	0.00	0.000000	0.000000	0.00	0.010000	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.050000	0.02	0.01	0.00	0.000000	0.01	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.01	0.00	0.00	0.00	0.00	0.01	0.00	0.00
20	Damansara Heights	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.02	0.020000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.010000	0.000000	0.00	0.000000	0.040000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.020000	0.00	0.00	0.00	0.00	0.030000	0.00	0.000000	0.00	0.01	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.060000	0.00	0.00	0.010000	0.00	0.00	0.000000	0.030000	0.00	0.01	0.01	0.01	0.030000	0.00	0.00	0.00	0.00	0.000000	0.00	0.02	0.00	0.01	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.01	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.020000	0.00	0.00	0.01	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.02	0.01	0.01	0.00	0.00	0.000000	0.010000	0.00	0.00	0.010000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.020000	0.00	0.00	0.000000	0.00	0.00	0.00	0.040000	0.050000	0.000000	0.03	0.00	0.010000	0.000000	0.00	0.00	0.000000	0.000000	0.01	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.030000	0.000000	0.00	0.01	0.01	0.00	0.00	0.01	0.000000	0.00	0.00	0.01	0.01	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.010000	0.000000	0.01	0.00	0.01	0.000000	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.000000	0.000000	0.01	0.01	0.010000	0.010000	0.000000	0.00	0.000000	0.000000	0.01	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.01	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.02	0.00	0.000000	0.000000	0.00	0.01	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.020000	0.01	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.020000	0.01	0.010000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.030000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.04	0.00	0.00	0.00	0.01	0.00
21	Damansara Town Centre	0.010000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.030000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.040000	0.030000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.01	0.01	0.010000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.040000	0.000000	0.000000	0.00	0.00	0.090000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.020000	0.00	0.01	0.00	0.01	0.020000	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.01	0.00	0.000000	0.01	0.00	0.00	0.040000	0.000000	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.000000	0.000000	0.00	0.01	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.030000	0.00	0.000000	0.00	0.00	0.00	0.01	0.010000	0.01	0.00	0.000000	0.000000	0.000000	0.00	0.01	0.00	0.00	0.02	0.00	0.010000	0.00	0.00	0.00	0.00	0.02	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.00	0.000000	0.00	0.00	0.00	0.030000	0.010000	0.000000	0.00	0.00	0.010000	0.000000	0.00	0.01	0.000000	0.000000	0.00	0.020000	0.00	0.010000	0.00	0.00	0.00	0.00	0.01	0.020000	0.000000	0.00	0.02	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.01	0.020000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.010000	0.00	0.00	0.01	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.01	0.000000	0.010000	0.00	0.01	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.01	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.020000	0.000000	0.01	0.000000	0.00	0.01	0.000000	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.01	0.00	0.03	0.00	0.000000	0.000000	0.00	0.00	0.020000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.01	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.010000	0.000000	0.00	0.020000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.030000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.010000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.01	0.01	0.00
22	Damansara, Kuala Lumpur	0.010000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.030000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.040000	0.030000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.01	0.01	0.010000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.040000	0.000000	0.000000	0.00	0.00	0.090000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.020000	0.00	0.01	0.00	0.01	0.020000	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.01	0.00	0.000000	0.01	0.00	0.00	0.040000	0.000000	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.000000	0.000000	0.00	0.01	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.030000	0.00	0.000000	0.00	0.00	0.00	0.01	0.010000	0.01	0.00	0.000000	0.000000	0.000000	0.00	0.01	0.00	0.00	0.02	0.00	0.010000	0.00	0.00	0.00	0.00	0.02	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.00	0.000000	0.00	0.00	0.00	0.030000	0.010000	0.000000	0.00	0.00	0.010000	0.000000	0.00	0.01	0.000000	0.000000	0.00	0.020000	0.00	0.010000	0.00	0.00	0.00	0.00	0.01	0.020000	0.000000	0.00	0.02	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.01	0.020000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.010000	0.00	0.00	0.01	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.01	0.000000	0.010000	0.00	0.01	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.01	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.020000	0.000000	0.01	0.000000	0.00	0.01	0.000000	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.01	0.00	0.03	0.00	0.000000	0.000000	0.00	0.00	0.020000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.01	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.010000	0.000000	0.00	0.020000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.030000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.010000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.01	0.01	0.00
23	Dang Wangi	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.01	0.000000	0.010000	0.00	0.00	0.000000	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.010000	0.00	0.000000	0.00	0.000000	0.01	0.00	0.00	0.010000	0.00	0.00	0.01	0.00	0.030000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.01	0.000000	0.000000	0.000000	0.00	0.00	0.030000	0.00	0.00	0.000000	0.00	0.01	0.000000	0.010000	0.00	0.00	0.00	0.03	0.020000	0.00	0.00	0.00	0.01	0.020000	0.01	0.01	0.01	0.00	0.000000	0.00	0.00	0.00	0.000000	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.000000	0.020000	0.00	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.020000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.000000	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.01	0.00	0.000000	0.140000	0.00	0.01	0.000000	0.00	0.00	0.01	0.000000	0.010000	0.010000	0.02	0.00	0.010000	0.000000	0.02	0.02	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.01	0.00	0.00	0.00	0.04	0.030000	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.000000	0.00	0.00	0.00	0.02	0.01	0.00	0.00	0.000000	0.00	0.00	0.010000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.020000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.000000	0.010000	0.000000	0.00	0.000000	0.00	0.01	0.000000	0.000000	0.00	0.00	0.01	0.01	0.010000	0.00	0.00	0.00	0.02	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.020000	0.00	0.01	0.010000	0.01	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.000000	0.00	0.010000	0.00	0.02	0.00	0.00	0.00	0.00	0.00	0.030000	0.00	0.00	0.00	0.000000	0.01	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.00
24	Desa Petaling	0.010000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.010000	0.00	0.00	0.090000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.030000	0.000000	0.00	0.010000	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.010000	0.01	0.00	0.000000	0.00	0.00	0.040000	0.000000	0.010000	0.00	0.00	0.030000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.150000	0.00	0.00	0.00	0.00	0.050000	0.00	0.00	0.01	0.00	0.040000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.00	0.01	0.01	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.020000	0.00	0.00	0.00	0.00	0.010000	0.010000	0.000000	0.010000	0.010000	0.00	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.02	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.020000	0.00	0.00	0.00	0.010000	0.010000	0.000000	0.00	0.00	0.040000	0.010000	0.00	0.00	0.010000	0.000000	0.00	0.010000	0.00	0.000000	0.00	0.00	0.01	0.00	0.01	0.020000	0.010000	0.01	0.01	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.010000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.01	0.00	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.020000	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.020000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.010000	0.000000	0.010000	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.010000	0.00	0.000000	0.000000	0.01	0.000000	0.01	0.00	0.00	0.00	0.00	0.01	0.00	0.010000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.020000	0.00	0.010000	0.000000	0.00	0.00	0.01	0.00	0.00	0.00	0.00	0.00
25	Federal Hill, Kuala Lumpur	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.01	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.01	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.050000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.040000	0.01	0.03	0.01	0.00	0.030000	0.00	0.00	0.00	0.00	0.020000	0.00	0.01	0.00	0.00	0.000000	0.00	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.000000	0.020000	0.000000	0.000000	0.010000	0.01	0.020000	0.00	0.00	0.02	0.00	0.000000	0.00	0.00	0.000000	0.020000	0.000000	0.00	0.00	0.00	0.00	0.00	0.01	0.000000	0.00	0.00	0.00	0.00	0.02	0.01	0.01	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.01	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.070000	0.02	0.00	0.000000	0.00	0.00	0.00	0.030000	0.080000	0.000000	0.01	0.00	0.020000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.01	0.00	0.01	0.030000	0.000000	0.00	0.00	0.01	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.000000	0.00	0.00	0.010000	0.01	0.000000	0.01	0.00	0.010000	0.000000	0.000000	0.020000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.020000	0.01	0.000000	0.000000	0.01	0.00	0.000000	0.000000	0.000000	0.00	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.010000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.01	0.00	0.000000	0.000000	0.00	0.01	0.020000	0.000000	0.000000	0.000000	0.00	0.02	0.010000	0.00	0.01	0.00	0.00	0.00	0.000000	0.000000	0.01	0.040000	0.00	0.000000	0.000000	0.00	0.010000	0.00	0.00	0.00	0.01	0.00	0.01	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.000000	0.000000	0.00	0.00	0.01	0.00	0.00	0.00	0.00	0.00
26	Happy Garden	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.01	0.00	0.070000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.02	0.010000	0.010000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.01	0.00	0.020000	0.00	0.030000	0.00	0.00	0.000000	0.00	0.00	0.030000	0.000000	0.000000	0.00	0.00	0.010000	0.00	0.00	0.000000	0.00	0.00	0.010000	0.110000	0.00	0.02	0.00	0.00	0.050000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.01	0.020000	0.000000	0.000000	0.00	0.00	0.00	0.01	0.00	0.00	0.01	0.00	0.00	0.00	0.00	0.010000	0.010000	0.00	0.00	0.00	0.00	0.010000	0.000000	0.010000	0.000000	0.010000	0.00	0.010000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.020000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.000000	0.020000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.00	0.010000	0.00	0.00	0.00	0.030000	0.000000	0.020000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.01	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.050000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.01	0.00	0.000000	0.010000	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.01	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.01	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.020000	0.00	0.00	0.020000	0.00	0.00	0.00	0.00	0.00	0.010000	0.000000	0.00	0.030000	0.00	0.000000	0.000000	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.040000	0.00	0.00	0.01	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.01	0.00	0.010000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00
27	Jalan Cochrane, Kuala Lumpur	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.050000	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.020000	0.020000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.020000	0.00	0.00	0.000000	0.00	0.00	0.030000	0.000000	0.000000	0.00	0.00	0.010000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.160000	0.00	0.00	0.00	0.00	0.040000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.020000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.010000	0.00	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.020000	0.00	0.01	0.00	0.01	0.010000	0.00	0.02	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.00	0.01	0.00	0.02	0.00	0.00	0.000000	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.030000	0.00	0.00	0.010000	0.00	0.00	0.00	0.030000	0.010000	0.000000	0.01	0.00	0.010000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.020000	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.040000	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.020000	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.000000	0.00	0.01	0.010000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.040000	0.030000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.010000	0.010000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.030000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.01	0.00	0.00	0.000000	0.00	0.00	0.00	0.02	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.010000	0.000000	0.00	0.020000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.020000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
28	Jalan Duta	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.010000	0.020000	0.00	0.00	0.050000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.010000	0.000000	0.00	0.030000	0.010000	0.00	0.010000	0.00	0.000000	0.00	0.01	0.00	0.020000	0.00	0.00	0.00	0.00	0.000000	0.00	0.000000	0.00	0.01	0.010000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.100000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.110000	0.00	0.00	0.00	0.00	0.030000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.010000	0.01	0.00	0.00	0.010000	0.000000	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.010000	0.000000	0.000000	0.000000	0.010000	0.01	0.000000	0.00	0.00	0.01	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.01	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.010000	0.01	0.00	0.010000	0.01	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.00	0.010000	0.040000	0.000000	0.00	0.00	0.030000	0.000000	0.00	0.01	0.000000	0.000000	0.00	0.010000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.01	0.00	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.010000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.020000	0.01	0.000000	0.000000	0.00	0.00	0.020000	0.000000	0.000000	0.01	0.010000	0.000000	0.01	0.00	0.00	0.00	0.00	0.01	0.00	0.000000	0.040000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.00	0.00	0.00	0.030000	0.00	0.00	0.00	0.02	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.000000	0.00	0.00	0.030000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.010000	0.000000	0.00	0.020000	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
29	Jinjang	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.150000	0.000000	0.00	0.00	0.000000	0.00	0.010000	0.000000	0.000000	0.00	0.020000	0.000000	0.00	0.010000	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.020000	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.040000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.250000	0.00	0.00	0.00	0.00	0.040000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.030000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.01	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.010000	0.000000	0.010000	0.000000	0.020000	0.00	0.050000	0.00	0.00	0.00	0.01	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.01	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.030000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.020000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.050000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.00	0.00	0.00	0.040000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.000000	0.00	0.01	0.010000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.010000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.070000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00
30	KL Eco City	0.010000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.060000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.020000	0.00	0.000000	0.00	0.00	0.010000	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.040000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.030000	0.00	0.01	0.00	0.00	0.020000	0.00	0.00	0.00	0.00	0.010000	0.00	0.02	0.00	0.00	0.000000	0.01	0.03	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.000000	0.010000	0.000000	0.010000	0.030000	0.00	0.000000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.020000	0.00	0.00	0.000000	0.00	0.00	0.00	0.070000	0.050000	0.000000	0.00	0.00	0.060000	0.000000	0.02	0.02	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.01	0.00	0.030000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.02	0.010000	0.00	0.000000	0.00	0.01	0.000000	0.000000	0.000000	0.010000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.010000	0.010000	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.010000	0.000000	0.00	0.000000	0.00	0.01	0.010000	0.000000	0.00	0.00	0.00	0.00	0.010000	0.01	0.02	0.00	0.05	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.010000	0.000000	0.000000	0.00	0.00	0.010000	0.00	0.00	0.00	0.01	0.00	0.000000	0.000000	0.00	0.020000	0.00	0.020000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.010000	0.01	0.00	0.00	0.000000	0.00	0.01	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.01	0.000000	0.010000	0.01	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
31	Kampung Baru, Kuala Lumpur	0.010000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.010000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.020000	0.010000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.010000	0.00	0.00	0.01	0.00	0.010000	0.00	0.010000	0.00	0.00	0.000000	0.00	0.01	0.000000	0.000000	0.000000	0.00	0.00	0.020000	0.00	0.01	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.07	0.00	0.02	0.020000	0.00	0.00	0.00	0.01	0.000000	0.01	0.04	0.01	0.00	0.000000	0.00	0.00	0.00	0.000000	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.020000	0.00	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.01	0.00	0.00	0.00	0.010000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.110000	0.02	0.01	0.000000	0.00	0.00	0.00	0.020000	0.000000	0.000000	0.02	0.00	0.010000	0.000000	0.02	0.04	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.02	0.040000	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.010000	0.02	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.020000	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.000000	0.010000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.00	0.00	0.01	0.010000	0.00	0.00	0.00	0.03	0.00	0.000000	0.000000	0.00	0.00	0.020000	0.000000	0.000000	0.000000	0.01	0.00	0.010000	0.00	0.00	0.01	0.00	0.00	0.000000	0.000000	0.00	0.030000	0.01	0.010000	0.000000	0.00	0.010000	0.00	0.01	0.00	0.00	0.00	0.00	0.01	0.020000	0.01	0.01	0.00	0.000000	0.01	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.00
32	Kampung Datuk Keramat	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.01	0.00	0.000000	0.040000	0.00	0.00	0.040000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.020000	0.010000	0.00	0.010000	0.00	0.000000	0.00	0.00	0.00	0.020000	0.00	0.00	0.01	0.00	0.000000	0.00	0.010000	0.00	0.00	0.000000	0.00	0.00	0.020000	0.000000	0.000000	0.00	0.00	0.040000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.01	0.080000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.010000	0.020000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.010000	0.00	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.010000	0.00	0.000000	0.00	0.00	0.01	0.00	0.000000	0.00	0.01	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.01	0.00	0.00	0.01	0.00	0.02	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.00	0.00	0.000000	0.070000	0.02	0.00	0.000000	0.00	0.00	0.00	0.010000	0.020000	0.010000	0.05	0.00	0.010000	0.000000	0.00	0.01	0.000000	0.010000	0.00	0.020000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.090000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.010000	0.000000	0.00	0.00	0.010000	0.010000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.000000	0.020000	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.000000	0.00	0.00	0.01	0.00	0.040000	0.00	0.00	0.00	0.01	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.01	0.00	0.000000	0.00	0.00	0.01	0.02	0.00	0.000000	0.000000	0.00	0.000000	0.01	0.020000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
33	Kepong	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.01	0.00	0.010000	0.000000	0.00	0.00	0.090000	0.000000	0.00	0.00	0.000000	0.01	0.000000	0.000000	0.000000	0.00	0.020000	0.000000	0.00	0.010000	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.030000	0.00	0.00	0.010000	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.030000	0.00	0.00	0.010000	0.00	0.00	0.000000	0.230000	0.00	0.00	0.00	0.00	0.060000	0.00	0.00	0.00	0.00	0.000000	0.00	0.01	0.00	0.00	0.010000	0.00	0.00	0.00	0.050000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.01	0.000000	0.000000	0.020000	0.000000	0.010000	0.00	0.030000	0.00	0.00	0.00	0.01	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.01	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.010000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.01	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.010000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.010000	0.000000	0.00	0.01	0.00	0.00	0.00	0.00	0.000000	0.00	0.01	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.050000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.020000	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.000000	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.030000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.020000	0.000000	0.000000	0.000000	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.020000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.030000	0.00	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
34	Kuchai Lama	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.040000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.030000	0.000000	0.00	0.030000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.020000	0.01	0.00	0.010000	0.00	0.00	0.020000	0.000000	0.000000	0.00	0.00	0.090000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.170000	0.00	0.00	0.00	0.00	0.030000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.01	0.00	0.000000	0.010000	0.000000	0.000000	0.000000	0.00	0.020000	0.00	0.00	0.00	0.00	0.000000	0.01	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.01	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.01	0.00	0.000000	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.00	0.01	0.00	0.010000	0.000000	0.010000	0.01	0.00	0.050000	0.000000	0.00	0.01	0.010000	0.000000	0.00	0.020000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.040000	0.000000	0.00	0.01	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.010000	0.030000	0.00	0.00	0.01	0.000000	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.010000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.01	0.01	0.000000	0.040000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.010000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.00	0.01	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.010000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.020000	0.00	0.000000	0.000000	0.00	0.01	0.01	0.00	0.00	0.00	0.00	0.00
35	Lembah Pantai	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.02	0.020000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.020000	0.010000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.020000	0.00	0.000000	0.00	0.00	0.010000	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.050000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.040000	0.00	0.04	0.00	0.00	0.050000	0.00	0.00	0.00	0.00	0.010000	0.00	0.02	0.00	0.00	0.000000	0.01	0.02	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.000000	0.010000	0.000000	0.010000	0.020000	0.00	0.000000	0.00	0.00	0.01	0.00	0.010000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.030000	0.00	0.00	0.000000	0.00	0.00	0.00	0.050000	0.050000	0.000000	0.00	0.00	0.030000	0.000000	0.01	0.02	0.000000	0.000000	0.00	0.000000	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.030000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.02	0.010000	0.00	0.000000	0.00	0.01	0.000000	0.000000	0.010000	0.010000	0.00	0.00	0.01	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.010000	0.010000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.000000	0.020000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.00	0.01	0.00	0.010000	0.00	0.00	0.00	0.05	0.00	0.000000	0.000000	0.00	0.01	0.010000	0.010000	0.000000	0.000000	0.00	0.01	0.020000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.020000	0.00	0.020000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.01	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.010000	0.000000	0.00	0.00	0.01	0.00	0.00	0.00	0.00	0.00
36	Maluri	0.000000	0.01	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.010000	0.00	0.00	0.000000	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.010000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.010000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.090000	0.00	0.00	0.000000	0.00	0.01	0.000000	0.040000	0.01	0.00	0.01	0.00	0.030000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.000000	0.010000	0.000000	0.000000	0.000000	0.00	0.020000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.01	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.000000	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.02	0.00	0.000000	0.090000	0.02	0.01	0.000000	0.00	0.00	0.01	0.000000	0.060000	0.000000	0.02	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.01	0.00	0.00	0.00	0.01	0.050000	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.010000	0.00	0.00	0.00	0.03	0.01	0.00	0.00	0.000000	0.00	0.00	0.010000	0.02	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.020000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.030000	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.030000	0.000000	0.00	0.000000	0.00	0.01	0.000000	0.000000	0.00	0.00	0.01	0.00	0.000000	0.00	0.01	0.00	0.01	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.020000	0.00	0.01	0.010000	0.01	0.02	0.00	0.01	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.01	0.00	0.01	0.00	0.01	0.00	0.010000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
37	Medan Tuanku	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.010000	0.00	0.00	0.020000	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.020000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.010000	0.00	0.00	0.01	0.00	0.010000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.01	0.000000	0.000000	0.000000	0.00	0.00	0.010000	0.00	0.00	0.000000	0.00	0.01	0.000000	0.030000	0.00	0.01	0.00	0.03	0.020000	0.00	0.00	0.00	0.01	0.020000	0.00	0.01	0.00	0.00	0.010000	0.00	0.00	0.00	0.000000	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.000000	0.010000	0.00	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.020000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.01	0.00	0.000000	0.120000	0.00	0.01	0.000000	0.00	0.00	0.01	0.000000	0.030000	0.010000	0.01	0.00	0.010000	0.000000	0.02	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.01	0.00	0.00	0.00	0.03	0.030000	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.010000	0.00	0.00	0.00	0.02	0.01	0.00	0.00	0.000000	0.00	0.00	0.010000	0.01	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.020000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.01	0.000000	0.020000	0.00	0.00	0.01	0.00	0.020000	0.00	0.01	0.00	0.02	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.020000	0.00	0.01	0.010000	0.01	0.01	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.02	0.00	0.00	0.00	0.00	0.00	0.050000	0.00	0.01	0.00	0.000000	0.01	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.01	0.00	0.00	0.00	0.00	0.01	0.00	0.00
38	Miharja	0.000000	0.01	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.010000	0.00	0.00	0.000000	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.010000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.010000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.090000	0.00	0.00	0.000000	0.00	0.01	0.000000	0.040000	0.01	0.00	0.01	0.00	0.030000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.000000	0.010000	0.000000	0.000000	0.000000	0.00	0.020000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.01	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.000000	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.02	0.00	0.000000	0.090000	0.02	0.01	0.000000	0.00	0.00	0.01	0.000000	0.060000	0.000000	0.02	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.01	0.00	0.00	0.00	0.01	0.050000	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.010000	0.00	0.00	0.00	0.03	0.01	0.00	0.00	0.000000	0.00	0.00	0.010000	0.02	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.020000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.030000	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.030000	0.000000	0.00	0.000000	0.00	0.01	0.000000	0.000000	0.00	0.00	0.01	0.00	0.000000	0.00	0.01	0.00	0.01	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.020000	0.00	0.01	0.010000	0.01	0.02	0.00	0.01	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.01	0.00	0.01	0.00	0.01	0.00	0.010000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
39	Mont Kiara	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.020000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.030000	0.020000	0.00	0.000000	0.00	0.000000	0.01	0.01	0.00	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.01	0.010000	0.000000	0.000000	0.01	0.00	0.130000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.00	0.00	0.000000	0.00	0.01	0.00	0.02	0.010000	0.00	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.000000	0.01	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.03	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.00	0.00	0.000000	0.010000	0.00	0.01	0.000000	0.00	0.00	0.00	0.030000	0.000000	0.010000	0.04	0.00	0.130000	0.000000	0.00	0.01	0.000000	0.000000	0.00	0.050000	0.01	0.000000	0.00	0.00	0.00	0.00	0.00	0.010000	0.000000	0.02	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.01	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.010000	0.01	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.010000	0.000000	0.01	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.040000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.030000	0.00	0.00	0.00	0.02	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.000000	0.00	0.00	0.030000	0.01	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.010000	0.00	0.010000	0.000000	0.00	0.010000	0.00	0.00	0.01	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.03	0.00
40	Pantai Dalam	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.030000	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.030000	0.000000	0.00	0.010000	0.010000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.01	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.020000	0.00	0.00	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.080000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.160000	0.00	0.01	0.00	0.00	0.020000	0.00	0.00	0.00	0.01	0.010000	0.00	0.00	0.00	0.00	0.000000	0.01	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.01	0.00	0.00	0.01	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.030000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.000000	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.00	0.00	0.000000	0.020000	0.00	0.00	0.000000	0.00	0.00	0.00	0.010000	0.050000	0.010000	0.00	0.00	0.050000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.010000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.050000	0.000000	0.00	0.01	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.010000	0.020000	0.00	0.00	0.01	0.000000	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.020000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.02	0.01	0.000000	0.030000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.020000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.01	0.00	0.000000	0.010000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.01	0.020000	0.00	0.00	0.00	0.01	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.010000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.010000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.000000	0.000000	0.00	0.01	0.00	0.00	0.00	0.00	0.00	0.00
41	Pudu, Kuala Lumpur	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.010000	0.010000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.01	0.00	0.020000	0.00	0.020000	0.00	0.00	0.000000	0.00	0.00	0.020000	0.000000	0.000000	0.00	0.00	0.030000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.160000	0.00	0.02	0.00	0.00	0.040000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.000000	0.00	0.01	0.00	0.020000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.000000	0.010000	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.01	0.00	0.00	0.010000	0.00	0.01	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.01	0.000000	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.01	0.000000	0.02	0.00	0.000000	0.080000	0.00	0.00	0.000000	0.00	0.00	0.00	0.010000	0.000000	0.000000	0.01	0.00	0.010000	0.000000	0.01	0.01	0.000000	0.000000	0.00	0.020000	0.00	0.000000	0.00	0.00	0.00	0.00	0.02	0.010000	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.02	0.000000	0.00	0.01	0.010000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.040000	0.030000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.010000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.040000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.01	0.00	0.00	0.000000	0.00	0.00	0.00	0.05	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.000000	0.00	0.00	0.020000	0.01	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.01	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.01	0.000000	0.020000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
42	Putrajaya	0.012821	0.00	0.00	0.00	0.012821	0.012821	0.00	0.00	0.00	0.000000	0.012821	0.00	0.00	0.064103	0.000000	0.00	0.00	0.012821	0.00	0.012821	0.000000	0.000000	0.00	0.012821	0.000000	0.00	0.000000	0.00	0.012821	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.012821	0.000000	0.000000	0.00	0.00	0.025641	0.00	0.00	0.000000	0.00	0.00	0.000000	0.102564	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.012821	0.00	0.00	0.00	0.00	0.012821	0.00	0.00	0.00	0.000000	0.000000	0.012821	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.012821	0.00	0.00	0.00	0.00	0.012821	0.012821	0.000000	0.000000	0.012821	0.00	0.051282	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.012821	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.012821	0.00	0.00	0.000000	0.00	0.012821	0.00	0.012821	0.00	0.00	0.012821	0.00	0.00	0.000000	0.012821	0.00	0.00	0.000000	0.00	0.00	0.00	0.012821	0.012821	0.025641	0.00	0.00	0.000000	0.012821	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.012821	0.00	0.00	0.00	0.00	0.00	0.076923	0.025641	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.012821	0.00	0.00	0.000000	0.00	0.012821	0.00	0.00	0.000000	0.012821	0.012821	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.000000	0.012821	0.00	0.00	0.025641	0.000000	0.012821	0.00	0.012821	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.025641	0.000000	0.00	0.012821	0.00	0.00	0.012821	0.012821	0.00	0.00	0.00	0.00	0.038462	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.012821	0.012821	0.025641	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.012821	0.00	0.00	0.000000	0.012821	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
43	Salak South	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.050000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.030000	0.000000	0.00	0.010000	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.000000	0.01	0.00	0.010000	0.00	0.00	0.030000	0.000000	0.010000	0.00	0.00	0.050000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.180000	0.00	0.00	0.00	0.00	0.060000	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.020000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.000000	0.010000	0.000000	0.010000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.01	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.030000	0.00	0.01	0.00	0.020000	0.010000	0.000000	0.01	0.00	0.040000	0.000000	0.00	0.00	0.010000	0.000000	0.00	0.030000	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.010000	0.010000	0.01	0.02	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.030000	0.00	0.01	0.00	0.000000	0.00	0.00	0.00	0.01	0.00	0.000000	0.00	0.010000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.000000	0.020000	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.010000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.010000	0.00	0.000000	0.000000	0.01	0.010000	0.01	0.00	0.00	0.00	0.00	0.01	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.010000	0.000000	0.00	0.01	0.02	0.00	0.00	0.00	0.00	0.00
44	Segambut	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.020000	0.00	0.00	0.050000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.010000	0.000000	0.00	0.020000	0.010000	0.00	0.010000	0.01	0.000000	0.01	0.01	0.00	0.010000	0.00	0.00	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.080000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.110000	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.010000	0.01	0.00	0.00	0.020000	0.000000	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.010000	0.01	0.010000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.01	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.02	0.00	0.00	0.000000	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.00	0.010000	0.010000	0.000000	0.01	0.00	0.040000	0.000000	0.00	0.01	0.000000	0.000000	0.00	0.060000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.050000	0.010000	0.01	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.010000	0.020000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.000000	0.030000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.00	0.02	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.000000	0.00	0.01	0.030000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.010000	0.000000	0.00	0.020000	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.010000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
45	Semarak	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.020000	0.01	0.01	0.070000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.020000	0.010000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.000000	0.01	0.00	0.00	0.00	0.000000	0.00	0.020000	0.00	0.00	0.000000	0.00	0.00	0.020000	0.000000	0.000000	0.00	0.00	0.010000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.080000	0.00	0.00	0.00	0.00	0.050000	0.00	0.00	0.01	0.00	0.010000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.00	0.010000	0.010000	0.00	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.020000	0.00	0.020000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.010000	0.01	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.000000	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.020000	0.00	0.00	0.000000	0.01	0.00	0.00	0.000000	0.030000	0.040000	0.01	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.180000	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.00	0.00	0.02	0.00	0.00	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.000000	0.000000	0.01	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.010000	0.00	0.000000	0.000000	0.00	0.00	0.030000	0.010000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.020000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.030000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.030000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00
46	Sentul Raya	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.010000	0.010000	0.00	0.00	0.050000	0.020000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.020000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.060000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.100000	0.00	0.03	0.00	0.00	0.040000	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.010000	0.000000	0.000000	0.000000	0.000000	0.01	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.020000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.040000	0.00	0.00	0.000000	0.00	0.00	0.01	0.010000	0.060000	0.010000	0.00	0.00	0.010000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.090000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.01	0.00	0.00	0.00	0.00	0.00	0.040000	0.00	0.01	0.010000	0.01	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.01	0.00	0.00	0.020000	0.01	0.000000	0.010000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.020000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.030000	0.00	0.00	0.00	0.01	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.020000	0.00	0.00	0.00	0.00	0.00	0.000000	0.010000	0.00	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.040000	0.02	0.00	0.00	0.000000	0.00	0.00	0.00	0.010000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.010000	0.01	0.00	0.00	0.00	0.00	0.00	0.00	0.00
47	Setapak	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.010000	0.010000	0.00	0.00	0.080000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.010000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.020000	0.00	0.010000	0.00	0.00	0.010000	0.00	0.00	0.010000	0.000000	0.010000	0.00	0.00	0.020000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.140000	0.00	0.00	0.00	0.00	0.050000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.020000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.010000	0.000000	0.00	0.00	0.00	0.00	0.010000	0.000000	0.010000	0.000000	0.010000	0.00	0.020000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.010000	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.00	0.000000	0.020000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.010000	0.00	0.00	0.000000	0.01	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.020000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.00	0.010000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.070000	0.000000	0.00	0.01	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.020000	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.020000	0.01	0.000000	0.000000	0.00	0.00	0.000000	0.010000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.040000	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.000000	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.010000	0.00	0.00	0.000000	0.00	0.00	0.00	0.01	0.00	0.000000	0.010000	0.00	0.010000	0.00	0.000000	0.000000	0.00	0.010000	0.02	0.00	0.00	0.00	0.00	0.00	0.00	0.030000	0.02	0.00	0.00	0.000000	0.00	0.00	0.00	0.010000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.020000	0.00	0.000000	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
48	Setiawangsa	0.000000	0.00	0.00	0.01	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.020000	0.00	0.00	0.080000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.020000	0.010000	0.00	0.010000	0.00	0.000000	0.00	0.00	0.00	0.010000	0.00	0.00	0.01	0.00	0.000000	0.00	0.020000	0.00	0.00	0.000000	0.00	0.00	0.030000	0.000000	0.000000	0.00	0.00	0.010000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.050000	0.00	0.00	0.00	0.00	0.050000	0.00	0.01	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.000000	0.01	0.00	0.00	0.000000	0.000000	0.010000	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.010000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.01	0.00	0.00	0.02	0.00	0.01	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.01	0.000000	0.000000	0.00	0.00	0.000000	0.01	0.00	0.00	0.010000	0.000000	0.030000	0.01	0.00	0.010000	0.000000	0.00	0.01	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.01	0.00	0.00	0.160000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.01	0.00	0.010000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.010000	0.00	0.020000	0.010000	0.00	0.00	0.010000	0.020000	0.000000	0.00	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.010000	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.020000	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.00	0.02	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.020000	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.020000	0.00	0.010000	0.000000	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.030000	0.00	0.00	0.00	0.000000	0.00	0.01	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00
49	Shamelin	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.050000	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.010000	0.00	0.00	0.01	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.050000	0.000000	0.000000	0.00	0.00	0.050000	0.00	0.00	0.010000	0.00	0.00	0.000000	0.170000	0.00	0.00	0.00	0.00	0.030000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.020000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.010000	0.01	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.010000	0.00	0.00	0.00	0.01	0.010000	0.00	0.02	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.01	0.00	0.01	0.00	0.00	0.000000	0.010000	0.00	0.00	0.000000	0.01	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.00	0.000000	0.00	0.00	0.00	0.040000	0.010000	0.000000	0.01	0.00	0.010000	0.000000	0.00	0.00	0.010000	0.000000	0.00	0.020000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.050000	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.000000	0.00	0.01	0.020000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.010000	0.010000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.000000	0.010000	0.00	0.00	0.010000	0.010000	0.000000	0.00	0.010000	0.000000	0.01	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.020000	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.020000	0.00	0.01	0.00	0.00	0.010000	0.00	0.00	0.00	0.02	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.010000	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.000000	0.000000	0.00	0.000000	0.00	0.010000	0.000000	0.00	0.020000	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.010000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
50	Sri Hartamas	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.020000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.030000	0.020000	0.00	0.000000	0.00	0.000000	0.01	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.01	0.010000	0.000000	0.000000	0.01	0.00	0.120000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.00	0.00	0.01	0.020000	0.00	0.00	0.00	0.00	0.000000	0.00	0.01	0.00	0.02	0.010000	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.000000	0.01	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.01	0.01	0.03	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.00	0.00	0.000000	0.010000	0.00	0.01	0.000000	0.00	0.00	0.00	0.030000	0.010000	0.010000	0.04	0.00	0.110000	0.000000	0.00	0.01	0.000000	0.000000	0.00	0.050000	0.01	0.000000	0.00	0.00	0.00	0.00	0.01	0.010000	0.000000	0.02	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.01	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.010000	0.01	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.000000	0.00	0.000000	0.000000	0.00	0.01	0.000000	0.000000	0.000000	0.00	0.010000	0.000000	0.01	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.040000	0.000000	0.00	0.000000	0.01	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.030000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.000000	0.00	0.00	0.030000	0.01	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.010000	0.00	0.010000	0.000000	0.00	0.010000	0.00	0.00	0.01	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.03	0.00
51	Sri Petaling	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.040000	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.010000	0.000000	0.00	0.020000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.010000	0.00	0.00	0.000000	0.00	0.00	0.030000	0.000000	0.010000	0.00	0.00	0.100000	0.00	0.00	0.010000	0.00	0.00	0.000000	0.140000	0.00	0.00	0.00	0.00	0.060000	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.01	0.00	0.01	0.00	0.000000	0.000000	0.000000	0.010000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.01	0.00	0.010000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.020000	0.00	0.01	0.00	0.020000	0.020000	0.000000	0.01	0.00	0.040000	0.000000	0.00	0.00	0.010000	0.000000	0.00	0.040000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.010000	0.000000	0.01	0.02	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.000000	0.010000	0.00	0.01	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.010000	0.010000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.000000	0.030000	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.010000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.01	0.010000	0.00	0.010000	0.000000	0.00	0.010000	0.01	0.00	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.030000	0.00	0.010000	0.000000	0.00	0.00	0.02	0.00	0.00	0.00	0.00	0.00
52	Sungai Besi	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.040000	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.010000	0.010000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.010000	0.00	0.000000	0.00	0.00	0.010000	0.00	0.00	0.030000	0.000000	0.000000	0.00	0.00	0.040000	0.00	0.00	0.000000	0.01	0.00	0.000000	0.060000	0.00	0.00	0.00	0.00	0.050000	0.00	0.00	0.00	0.00	0.040000	0.00	0.00	0.00	0.00	0.000000	0.00	0.01	0.00	0.010000	0.010000	0.000000	0.00	0.00	0.00	0.01	0.00	0.01	0.00	0.01	0.00	0.00	0.00	0.000000	0.060000	0.01	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.010000	0.00	0.020000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.01	0.00	0.00	0.02	0.02	0.00	0.00	0.000000	0.030000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.01	0.00	0.000000	0.00	0.00	0.000000	0.030000	0.00	0.00	0.000000	0.01	0.00	0.00	0.010000	0.020000	0.020000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.01	0.00	0.00	0.090000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.040000	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.020000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.010000	0.00	0.010000	0.000000	0.00	0.00	0.02	0.00	0.00	0.00	0.00	0.010000	0.020000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.030000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.01	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.010000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.010000	0.00	0.000000	0.00	0.000000	0.000000	0.01	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.01	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
53	Taman Bukit Maluri	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.010000	0.000000	0.00	0.00	0.100000	0.000000	0.00	0.00	0.000000	0.01	0.000000	0.000000	0.000000	0.00	0.020000	0.000000	0.00	0.020000	0.00	0.000000	0.00	0.00	0.00	0.010000	0.00	0.00	0.01	0.00	0.000000	0.00	0.010000	0.00	0.00	0.000000	0.00	0.01	0.010000	0.000000	0.000000	0.00	0.00	0.060000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.180000	0.00	0.00	0.00	0.00	0.030000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.050000	0.000000	0.000000	0.00	0.00	0.01	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.000000	0.000000	0.010000	0.020000	0.020000	0.00	0.030000	0.00	0.00	0.00	0.02	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.01	0.00	0.00	0.010000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.00	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.040000	0.000000	0.00	0.01	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.02	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.000000	0.01	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.020000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.030000	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.010000	0.00	0.000000	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.010000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.000000	0.00	0.00	0.010000	0.01	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.040000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.030000	0.00	0.010000	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00
54	Taman Cheras Hartamas	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.010000	0.000000	0.00	0.010000	0.010000	0.00	0.010000	0.00	0.000000	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.000000	0.00	0.010000	0.00	0.00	0.010000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.070000	0.00	0.00	0.010000	0.00	0.00	0.010000	0.180000	0.00	0.00	0.00	0.00	0.040000	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.00	0.00	0.000000	0.01	0.00	0.00	0.000000	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.010000	0.00	0.020000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.000000	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.020000	0.00	0.00	0.00	0.020000	0.030000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.01	0.000000	0.010000	0.00	0.010000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.040000	0.000000	0.00	0.01	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.010000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.060000	0.00	0.00	0.00	0.010000	0.00	0.01	0.00	0.00	0.00	0.030000	0.00	0.010000	0.010000	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.000000	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.000000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.000000	0.010000	0.00	0.00	0.020000	0.000000	0.000000	0.000000	0.00	0.00	0.040000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.020000	0.00	0.000000	0.000000	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.040000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.010000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
55	Taman Connaught	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.020000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.010000	0.000000	0.00	0.010000	0.010000	0.00	0.010000	0.00	0.000000	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.000000	0.00	0.020000	0.00	0.00	0.020000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.040000	0.00	0.00	0.010000	0.00	0.00	0.010000	0.200000	0.00	0.00	0.00	0.00	0.040000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.020000	0.000000	0.010000	0.000000	0.010000	0.00	0.010000	0.00	0.00	0.00	0.01	0.010000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.020000	0.00	0.00	0.00	0.020000	0.010000	0.000000	0.00	0.00	0.010000	0.000000	0.00	0.01	0.000000	0.010000	0.00	0.010000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.050000	0.000000	0.00	0.01	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.010000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.000000	0.020000	0.00	0.00	0.00	0.010000	0.00	0.01	0.00	0.00	0.00	0.010000	0.00	0.010000	0.010000	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.000000	0.010000	0.01	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.000000	0.00	0.00	0.00	0.00	0.030000	0.00	0.00	0.00	0.00	0.00	0.000000	0.010000	0.00	0.00	0.020000	0.000000	0.000000	0.000000	0.00	0.00	0.040000	0.00	0.00	0.00	0.01	0.00	0.000000	0.000000	0.00	0.020000	0.00	0.000000	0.000000	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.040000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.010000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
56	Taman Desa	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.030000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.020000	0.000000	0.00	0.060000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.010000	0.00	0.00	0.020000	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.070000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.080000	0.00	0.01	0.00	0.00	0.020000	0.00	0.00	0.00	0.00	0.000000	0.00	0.01	0.00	0.00	0.000000	0.00	0.01	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.010000	0.020000	0.00	0.020000	0.00	0.00	0.00	0.00	0.010000	0.01	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.01	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.00	0.01	0.00	0.050000	0.000000	0.000000	0.01	0.00	0.070000	0.000000	0.00	0.03	0.000000	0.000000	0.00	0.010000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.040000	0.000000	0.00	0.01	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.010000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.010000	0.020000	0.00	0.00	0.01	0.000000	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.010000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.000000	0.020000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.010000	0.00	0.01	0.00	0.02	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.00	0.02	0.00	0.000000	0.000000	0.00	0.010000	0.00	0.010000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.040000	0.00	0.00	0.00	0.000000	0.00	0.01	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.01	0.000000	0.020000	0.00	0.010000	0.000000	0.00	0.01	0.01	0.00	0.00	0.00	0.00	0.00
57	Taman Ibukota	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.01	0.00	0.000000	0.000000	0.01	0.00	0.030000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.010000	0.000000	0.01	0.000000	0.010000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.01	0.00	0.020000	0.00	0.020000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.010000	0.00	0.01	0.000000	0.00	0.00	0.000000	0.00	0.00	0.010000	0.080000	0.00	0.01	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.01	0.010000	0.000000	0.010000	0.00	0.00	0.00	0.01	0.00	0.00	0.01	0.00	0.00	0.00	0.01	0.000000	0.020000	0.00	0.00	0.00	0.00	0.010000	0.000000	0.020000	0.000000	0.020000	0.01	0.010000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.01	0.00	0.00	0.000000	0.020000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.010000	0.010000	0.00	0.00	0.010000	0.00	0.00	0.00	0.020000	0.000000	0.040000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.01	0.000000	0.00	0.000000	0.00	0.00	0.01	0.00	0.00	0.150000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.01	0.00	0.000000	0.010000	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.000000	0.030000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.01	0.00	0.000000	0.00	0.00	0.01	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.010000	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.010000	0.000000	0.00	0.030000	0.00	0.000000	0.000000	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.040000	0.00	0.00	0.01	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
58	Taman Len Seng	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.030303	0.010101	0.00	0.00	0.000000	0.00	0.000000	0.010101	0.010101	0.00	0.010101	0.000000	0.00	0.010101	0.00	0.000000	0.00	0.00	0.00	0.020202	0.00	0.00	0.00	0.00	0.000000	0.00	0.010101	0.00	0.00	0.010101	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.040404	0.00	0.00	0.010101	0.00	0.00	0.010101	0.121212	0.00	0.00	0.00	0.00	0.020202	0.00	0.00	0.00	0.00	0.040404	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.010101	0.030303	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.010101	0.000000	0.00	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.010101	0.020202	0.00	0.020202	0.00	0.00	0.00	0.00	0.010101	0.00	0.00	0.000000	0.010101	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.010101	0.020202	0.00	0.00	0.010101	0.00	0.000000	0.00	0.000000	0.00	0.00	0.010101	0.00	0.00	0.000000	0.000000	0.00	0.00	0.010101	0.00	0.00	0.00	0.000000	0.030303	0.000000	0.00	0.00	0.010101	0.000000	0.00	0.00	0.000000	0.010101	0.00	0.010101	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.060606	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.010101	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.010101	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.060606	0.00	0.00	0.00	0.010101	0.00	0.00	0.00	0.00	0.00	0.010101	0.00	0.010101	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.010101	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.010101	0.030303	0.010101	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.010101	0.00	0.00	0.00	0.00	0.00	0.010101	0.010101	0.00	0.00	0.010101	0.010101	0.000000	0.000000	0.00	0.00	0.010101	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.020202	0.00	0.010101	0.010101	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.020202	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.010101	0.00	0.000000	0.00	0.00	0.000000	0.020202	0.00	0.010101	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
59	Taman Melati	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.01	0.00	0.000000	0.000000	0.00	0.00	0.070000	0.000000	0.00	0.00	0.000000	0.01	0.000000	0.010000	0.000000	0.00	0.010000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.010000	0.00	0.00	0.000000	0.01	0.00	0.030000	0.000000	0.000000	0.00	0.00	0.020000	0.01	0.00	0.000000	0.00	0.00	0.010000	0.020000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.01	0.00	0.030000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.01	0.000000	0.000000	0.020000	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.010000	0.040000	0.00	0.00	0.00	0.00	0.010000	0.000000	0.000000	0.000000	0.010000	0.01	0.010000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.02	0.02	0.00	0.00	0.000000	0.020000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.010000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.010000	0.050000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.01	0.000000	0.00	0.000000	0.00	0.00	0.01	0.00	0.00	0.150000	0.000000	0.00	0.01	0.00	0.00	0.00	0.00	0.040000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.02	0.00	0.000000	0.010000	0.000000	0.010000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.000000	0.000000	0.00	0.00	0.020000	0.000000	0.010000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.000000	0.020000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.01	0.00	0.01	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.01	0.00	0.000000	0.010000	0.000000	0.010000	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.060000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
60	Taman Midah	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.010000	0.00	0.00	0.060000	0.000000	0.01	0.00	0.000000	0.00	0.000000	0.010000	0.000000	0.00	0.000000	0.010000	0.00	0.010000	0.00	0.000000	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.000000	0.00	0.040000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.040000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.190000	0.00	0.00	0.00	0.00	0.040000	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.010000	0.000000	0.010000	0.000000	0.000000	0.00	0.020000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.00	0.00	0.00	0.030000	0.010000	0.010000	0.00	0.00	0.010000	0.000000	0.00	0.01	0.000000	0.010000	0.00	0.030000	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.040000	0.010000	0.00	0.01	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.01	0.00	0.000000	0.000000	0.000000	0.020000	0.00	0.00	0.00	0.000000	0.00	0.01	0.00	0.01	0.00	0.000000	0.00	0.010000	0.000000	0.00	0.00	0.020000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.020000	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.010000	0.00	0.00	0.00	0.00	0.040000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.000000	0.00	0.00	0.040000	0.00	0.00	0.00	0.01	0.00	0.000000	0.000000	0.00	0.010000	0.00	0.000000	0.000000	0.01	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
61	Taman OUG	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.01	0.00	0.010000	0.000000	0.00	0.00	0.100000	0.000000	0.00	0.00	0.000000	0.01	0.000000	0.000000	0.000000	0.00	0.020000	0.000000	0.00	0.010000	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.020000	0.00	0.00	0.020000	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.040000	0.00	0.00	0.010000	0.00	0.00	0.000000	0.250000	0.00	0.00	0.00	0.00	0.040000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.050000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.01	0.000000	0.000000	0.010000	0.010000	0.020000	0.00	0.030000	0.00	0.00	0.00	0.01	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.01	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.010000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.01	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.01	0.00	0.00	0.00	0.00	0.000000	0.00	0.01	0.00	0.00	0.00	0.01	0.00	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.030000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.020000	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.000000	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.040000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.020000	0.000000	0.000000	0.000000	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.020000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.030000	0.00	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
62	Taman P. Ramlee	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.010753	0.000000	0.00	0.00	0.043011	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.010753	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.010753	0.00	0.00	0.00	0.00	0.021505	0.00	0.010753	0.00	0.00	0.000000	0.00	0.00	0.010753	0.010753	0.010753	0.00	0.00	0.010753	0.00	0.00	0.000000	0.00	0.00	0.000000	0.139785	0.00	0.00	0.00	0.00	0.043011	0.00	0.00	0.00	0.00	0.010753	0.00	0.00	0.00	0.00	0.021505	0.00	0.00	0.00	0.010753	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.010753	0.000000	0.00	0.00	0.00	0.00	0.010753	0.000000	0.021505	0.000000	0.032258	0.00	0.010753	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.010753	0.010753	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.021505	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.021505	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.010753	0.010753	0.00	0.00	0.010753	0.00	0.00	0.00	0.000000	0.021505	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.010753	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.096774	0.010753	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.021505	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.010753	0.021505	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.021505	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.010753	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.032258	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.010753	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.010753	0.000000	0.000000	0.010753	0.00	0.00	0.010753	0.00	0.00	0.00	0.00	0.00	0.010753	0.010753	0.00	0.021505	0.00	0.000000	0.000000	0.00	0.010753	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.032258	0.00	0.00	0.00	0.010753	0.00	0.00	0.00	0.010753	0.000000	0.00	0.000000	0.00	0.00	0.010753	0.010753	0.00	0.000000	0.010753	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
63	Taman Sri Sinar	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.060000	0.000000	0.00	0.00	0.000000	0.00	0.010000	0.000000	0.000000	0.00	0.030000	0.010000	0.00	0.000000	0.01	0.000000	0.02	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.010000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.150000	0.00	0.00	0.00	0.00	0.030000	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.020000	0.000000	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.020000	0.010000	0.00	0.050000	0.00	0.00	0.01	0.01	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.01	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.000000	0.010000	0.01	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.020000	0.000000	0.02	0.00	0.030000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.080000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.030000	0.010000	0.01	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.010000	0.01	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.010000	0.010000	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.01	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.050000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.01	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.01	0.020000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.010000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.050000	0.00	0.01	0.01	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00
64	Taman Taynton View	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.040000	0.000000	0.01	0.00	0.000000	0.00	0.000000	0.010000	0.000000	0.00	0.020000	0.010000	0.00	0.010000	0.00	0.000000	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.000000	0.00	0.020000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.060000	0.00	0.00	0.010000	0.00	0.00	0.010000	0.200000	0.00	0.00	0.00	0.00	0.040000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.010000	0.000000	0.010000	0.000000	0.000000	0.00	0.020000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.030000	0.00	0.00	0.00	0.020000	0.010000	0.000000	0.00	0.00	0.010000	0.000000	0.00	0.01	0.000000	0.010000	0.00	0.010000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.020000	0.000000	0.00	0.01	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.010000	0.00	0.000000	0.01	0.00	0.000000	0.000000	0.000000	0.040000	0.00	0.00	0.00	0.000000	0.00	0.01	0.00	0.00	0.00	0.000000	0.00	0.010000	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.000000	0.020000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.020000	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.000000	0.00	0.00	0.00	0.00	0.040000	0.00	0.00	0.00	0.00	0.00	0.000000	0.010000	0.00	0.00	0.020000	0.000000	0.000000	0.000000	0.00	0.00	0.040000	0.00	0.00	0.00	0.01	0.00	0.000000	0.000000	0.00	0.010000	0.00	0.000000	0.000000	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.030000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.010000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
65	Taman Tun Dr Ismail	0.010000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.01	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.020000	0.000000	0.00	0.030000	0.010000	0.01	0.000000	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.020000	0.00	0.000000	0.00	0.00	0.010000	0.00	0.01	0.020000	0.000000	0.000000	0.00	0.00	0.010000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.020000	0.00	0.05	0.00	0.00	0.060000	0.00	0.01	0.01	0.00	0.010000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.010000	0.000000	0.020000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00	0.000000	0.010000	0.00	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.010000	0.00	0.010000	0.00	0.00	0.00	0.00	0.010000	0.00	0.01	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.01	0.00	0.01	0.00	0.00	0.01	0.00	0.00	0.000000	0.000000	0.00	0.01	0.010000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.00	0.00	0.000000	0.00	0.00	0.00	0.020000	0.000000	0.000000	0.00	0.00	0.020000	0.000000	0.00	0.02	0.000000	0.000000	0.00	0.020000	0.00	0.010000	0.00	0.00	0.00	0.00	0.01	0.040000	0.000000	0.01	0.01	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.040000	0.00	0.000000	0.01	0.00	0.000000	0.000000	0.010000	0.010000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.02	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.060000	0.000000	0.01	0.000000	0.00	0.01	0.000000	0.000000	0.00	0.01	0.00	0.00	0.000000	0.00	0.02	0.00	0.03	0.00	0.000000	0.000000	0.01	0.00	0.030000	0.000000	0.000000	0.000000	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.01	0.000000	0.000000	0.00	0.010000	0.00	0.000000	0.000000	0.00	0.020000	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.010000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.010000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
66	Taman U-Thant	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.01	0.000000	0.010000	0.00	0.00	0.010000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.020000	0.010000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.010000	0.00	0.00	0.01	0.00	0.020000	0.00	0.010000	0.00	0.00	0.000000	0.00	0.01	0.000000	0.000000	0.000000	0.00	0.00	0.020000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.03	0.00	0.02	0.030000	0.00	0.00	0.00	0.01	0.000000	0.01	0.03	0.01	0.00	0.000000	0.00	0.00	0.00	0.000000	0.020000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.000000	0.00	0.000000	0.01	0.00	0.01	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.01	0.00	0.02	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.110000	0.03	0.00	0.000000	0.00	0.00	0.00	0.000000	0.010000	0.000000	0.03	0.00	0.020000	0.000000	0.02	0.05	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.05	0.010000	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.020000	0.01	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.010000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.020000	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.02	0.000000	0.010000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.01	0.000000	0.00	0.00	0.00	0.05	0.00	0.000000	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.000000	0.01	0.00	0.000000	0.00	0.00	0.01	0.00	0.00	0.000000	0.000000	0.00	0.010000	0.01	0.020000	0.000000	0.00	0.010000	0.00	0.01	0.00	0.01	0.00	0.00	0.01	0.020000	0.00	0.00	0.00	0.000000	0.00	0.00	0.01	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.00
67	Taman Wahyu	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.01	0.00	0.00	0.000000	0.000000	0.00	0.00	0.070000	0.010000	0.00	0.01	0.000000	0.00	0.000000	0.000000	0.000000	0.00	0.010000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.020000	0.00	0.00	0.00	0.00	0.000000	0.00	0.010000	0.01	0.00	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.010000	0.170000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.030000	0.00	0.00	0.00	0.00	0.000000	0.00	0.01	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.01	0.00	0.00	0.00	0.00	0.010000	0.020000	0.00	0.00	0.00	0.00	0.010000	0.000000	0.000000	0.010000	0.010000	0.01	0.020000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.02	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.010000	0.00	0.00	0.000000	0.010000	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.030000	0.010000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.01	0.140000	0.010000	0.00	0.01	0.00	0.00	0.00	0.01	0.000000	0.01	0.00	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.000000	0.010000	0.00	0.01	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.030000	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.020000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.01	0.00	0.000000	0.020000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.010000	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.010000	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.030000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.01	0.000000	0.00	0.00	0.000000	0.020000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.01	0.00
68	Titiwangsa	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.010000	0.010000	0.00	0.00	0.080000	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.010000	0.000000	0.00	0.020000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.040000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.070000	0.00	0.02	0.00	0.00	0.020000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.010000	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.010000	0.000000	0.010000	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.030000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.010000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.030000	0.00	0.00	0.000000	0.00	0.00	0.00	0.010000	0.050000	0.020000	0.00	0.00	0.010000	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.090000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.01	0.00	0.00	0.01	0.00	0.00	0.040000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.020000	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.01	0.00	0.00	0.020000	0.01	0.000000	0.010000	0.00	0.00	0.000000	0.010000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.030000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.040000	0.00	0.00	0.00	0.01	0.00	0.000000	0.000000	0.01	0.00	0.000000	0.000000	0.000000	0.000000	0.00	0.00	0.010000	0.00	0.00	0.00	0.00	0.00	0.000000	0.010000	0.00	0.020000	0.01	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.050000	0.02	0.00	0.00	0.000000	0.00	0.00	0.00	0.010000	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.000000	0.010000	0.01	0.00	0.00	0.00	0.00	0.00	0.00	0.00
69	Wangsa Maju	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.01	0.00	0.070000	0.020000	0.00	0.00	0.000000	0.00	0.000000	0.000000	0.010000	0.02	0.010000	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.02	0.00	0.010000	0.01	0.000000	0.00	0.00	0.010000	0.00	0.00	0.020000	0.000000	0.000000	0.00	0.00	0.040000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.050000	0.00	0.01	0.00	0.00	0.020000	0.00	0.00	0.01	0.00	0.030000	0.00	0.01	0.00	0.00	0.000000	0.00	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.02	0.00	0.00	0.00	0.00	0.000000	0.030000	0.00	0.01	0.00	0.00	0.010000	0.000000	0.000000	0.000000	0.010000	0.00	0.040000	0.00	0.00	0.00	0.01	0.000000	0.00	0.00	0.010000	0.000000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.01	0.01	0.01	0.00	0.00	0.000000	0.010000	0.00	0.00	0.000000	0.00	0.000000	0.00	0.000000	0.00	0.00	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.000000	0.00	0.00	0.00	0.040000	0.010000	0.020000	0.00	0.00	0.000000	0.000000	0.00	0.01	0.000000	0.000000	0.00	0.010000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.070000	0.010000	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.020000	0.00	0.000000	0.00	0.00	0.000000	0.010000	0.000000	0.000000	0.00	0.00	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.000000	0.00	0.000000	0.030000	0.00	0.00	0.000000	0.000000	0.000000	0.00	0.010000	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.000000	0.020000	0.000000	0.01	0.000000	0.00	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.010000	0.00	0.00	0.00	0.01	0.00	0.000000	0.000000	0.02	0.00	0.010000	0.000000	0.000000	0.000000	0.00	0.00	0.020000	0.00	0.00	0.00	0.00	0.00	0.000000	0.000000	0.00	0.020000	0.01	0.000000	0.000000	0.00	0.000000	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.020000	0.00	0.00	0.01	0.000000	0.00	0.01	0.00	0.000000	0.000000	0.00	0.000000	0.01	0.00	0.000000	0.000000	0.00	0.000000	0.000000	0.00	0.00	0.00	0.00	0.01	0.00	0.00	0.00

In [ ]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

39

<h2>Create a new DataFrame for Shopping Mall data only</h2>

In [ ]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

In [ ]:
kl_mall.head()

       Neighborhoods             Shopping Mall
0      Alam Damai                0.00
1      Ampang, Kuala Lumpur      0.00
2      Bandar Menjalara          0.00
3      Bandar Sri Permaisuri     0.00
4      Bandar Tasik Selatan      0.01

<h2>7. Cluster Neighborhoods</h2>

Run k-means to cluster the neighborhoods in Kuala Lumpur into 3 clusters.

In [ ]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
array([1, 1, 1, 1, 1, 1, 2, 2, 0, 1])

In [ ]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [ ]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

In [ ]:
Neighborhood	Shopping Mall	Cluster Labels
0	Alam Damai	0.00	1
1	Ampang, Kuala Lumpur	0.00	1
2	Bandar Menjalara	0.00	1
3	Bandar Sri Permaisuri	0.00	1
4	Bandar Tasik Selatan	0.01	1

In [ ]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(70, 5)

In [ ]:
Neighborhood	Shopping Mall	Cluster Labels	Latitude	Longitude
0	Alam Damai	0.00	1	3.057690	101.743880
1	Ampang, Kuala Lumpur	0.00	1	3.158093	101.748703
2	Bandar Menjalara	0.00	1	3.190350	101.625450
3	Bandar Sri Permaisuri	0.00	1	3.103910	101.712260
4	Bandar Tasik Selatan	0.01	1	3.072620	101.714710

In [ ]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(70, 5)

In [ ]:
Neighborhood	Shopping Mall	Cluster Labels	Latitude	Longitude
19	Chow Kit	0.03	0	3.163590	101.698110
37	Medan Tuanku	0.02	0	3.159260	101.698340
44	Segambut	0.02	0	3.186390	101.668100
31	Kampung Baru, Kuala Lumpur	0.03	0	3.165460	101.710280
48	Setiawangsa	0.02	0	3.178434	101.737013
49	Shamelin	0.02	0	3.124580	101.735970
28	Jalan Duta	0.02	0	3.179388	101.677454
27	Jalan Cochrane, Kuala Lumpur	0.02	0	3.132962	101.724703
23	Dang Wangi	0.02	0	3.155238	101.701520
22	Damansara, Kuala Lumpur	0.03	0	3.141316	101.626080
21	Damansara Town Centre	0.03	0	3.141316	101.626080
20	Damansara Heights	0.02	0	3.147980	101.667980
17	Bukit Tunku	0.02	0	3.173810	101.682760
15	Bukit Nanas	0.03	0	3.152008	101.701028
39	Mont Kiara	0.02	0	3.165320	101.652430
56	Taman Desa	0.02	0	3.102970	101.684710
12	Bukit Bintang	0.03	0	3.147770	101.708550
65	Taman Tun Dr Ismail	0.03	0	3.152830	101.622710
8	Bangsar South	0.03	0	3.111020	101.662830
67	Taman Wahyu	0.00	1	3.222400	101.671730
43	Salak South	0.00	1	3.081020	101.697240
45	Semarak	0.00	1	3.180393	101.723414
46	Sentul Raya	0.01	1	3.175375	101.693034
47	Setapak	0.00	1	3.188160	101.704150
64	Taman Taynton View	0.00	1	3.087070	101.736810
63	Taman Sri Sinar	0.01	1	3.190070	101.652930
62	Taman P. Ramlee	0.00	1	3.193600	101.705980
61	Taman OUG	0.00	1	3.210037	101.634498
60	Taman Midah	0.00	1	3.093590	101.728370
50	Sri Hartamas	0.00	1	3.162200	101.650360
51	Sri Petaling	0.00	1	3.072600	101.682520
52	Sungai Besi	0.01	1	3.050640	101.706130
59	Taman Melati	0.00	1	3.223570	101.723990
58	Taman Len Seng	0.00	1	3.069080	101.742870
57	Taman Ibukota	0.00	1	3.212160	101.715400
42	Putrajaya	0.00	1	3.111867	101.713268
53	Taman Bukit Maluri	0.00	1	3.200660	101.633370
54	Taman Cheras Hartamas	0.00	1	3.082630	101.746710
55	Taman Connaught	0.00	1	3.082690	101.736890
0	Alam Damai	0.00	1	3.057690	101.743880
34	Kuchai Lama	0.00	1	3.090740	101.677330
14	Bukit Kiara	0.00	1	3.143480	101.644330
1	Ampang, Kuala Lumpur	0.00	1	3.158093	101.748703
2	Bandar Menjalara	0.00	1	3.190350	101.625450
3	Bandar Sri Permaisuri	0.00	1	3.103910	101.712260
4	Bandar Tasik Selatan	0.01	1	3.072620	101.714710
5	Bandar Tun Razak	0.00	1	3.082800	101.722810
9	Batu 11 Cheras	0.00	1	3.098980	101.734990
10	Batu, Kuala Lumpur	0.00	1	3.098980	101.734990
13	Bukit Jalil	0.00	1	3.057800	101.689650
40	Pantai Dalam	0.01	1	3.095160	101.668120
16	Bukit Petaling	0.01	1	3.129290	101.698920
18	Cheras, Kuala Lumpur	0.00	1	3.061870	101.746750
69	Wangsa Maju	0.01	1	3.203910	101.737190
25	Federal Hill, Kuala Lumpur	0.01	1	3.136370	101.685640
26	Happy Garden	0.01	1	3.201630	101.721070
29	Jinjang	0.00	1	3.209500	101.658740
32	Kampung Datuk Keramat	0.01	1	3.166400	101.730460
33	Kepong	0.00	1	3.217500	101.637630
68	Titiwangsa	0.01	1	3.180670	101.703220
36	Maluri	0.01	1	3.147890	101.694050
38	Miharja	0.01	1	3.147890	101.694050
24	Desa Petaling	0.00	1	3.083310	101.704380
11	Brickfields	0.05	2	3.129160	101.684060
30	KL Eco City	0.05	2	3.117110	101.673490
7	Bangsar Park	0.04	2	3.134780	101.672620
6	Bangsar	0.05	2	3.129200	101.678440
35	Lembah Pantai	0.05	2	3.121216	101.663897
66	Taman U-Thant	0.05	2	3.157650	101.725020
41	Pudu, Kuala Lumpur	0.05	2	3.133540	101.713070

<h2>Finally, let's visualize the resulting clusters</h2>

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

<h2>8. Examine Clusters</h2>

<h2>Cluster 0</h2>

In [ ]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

In [ ]:
Neighborhood	Shopping Mall	Cluster Labels	Latitude	Longitude
19	Chow Kit	0.03	0	3.163590	101.698110
37	Medan Tuanku	0.02	0	3.159260	101.698340
44	Segambut	0.02	0	3.186390	101.668100
31	Kampung Baru, Kuala Lumpur	0.03	0	3.165460	101.710280
48	Setiawangsa	0.02	0	3.178434	101.737013
49	Shamelin	0.02	0	3.124580	101.735970
28	Jalan Duta	0.02	0	3.179388	101.677454
27	Jalan Cochrane, Kuala Lumpur	0.02	0	3.132962	101.724703
23	Dang Wangi	0.02	0	3.155238	101.701520
22	Damansara, Kuala Lumpur	0.03	0	3.141316	101.626080
21	Damansara Town Centre	0.03	0	3.141316	101.626080
20	Damansara Heights	0.02	0	3.147980	101.667980
17	Bukit Tunku	0.02	0	3.173810	101.682760
15	Bukit Nanas	0.03	0	3.152008	101.701028
39	Mont Kiara	0.02	0	3.165320	101.652430
56	Taman Desa	0.02	0	3.102970	101.684710
12	Bukit Bintang	0.03	0	3.147770	101.708550
65	Taman Tun Dr Ismail	0.03	0	3.152830	101.622710
8	Bangsar South	0.03	0	3.111020	101.662830

<h2>Cluster 1</h2>

In [ ]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

In [ ]:
Neighborhood	Shopping Mall	Cluster Labels	Latitude	Longitude
67	Taman Wahyu	0.00	1	3.222400	101.671730
43	Salak South	0.00	1	3.081020	101.697240
45	Semarak	0.00	1	3.180393	101.723414
46	Sentul Raya	0.01	1	3.175375	101.693034
47	Setapak	0.00	1	3.188160	101.704150
64	Taman Taynton View	0.00	1	3.087070	101.736810
63	Taman Sri Sinar	0.01	1	3.190070	101.652930
62	Taman P. Ramlee	0.00	1	3.193600	101.705980
61	Taman OUG	0.00	1	3.210037	101.634498
60	Taman Midah	0.00	1	3.093590	101.728370
50	Sri Hartamas	0.00	1	3.162200	101.650360
51	Sri Petaling	0.00	1	3.072600	101.682520
52	Sungai Besi	0.01	1	3.050640	101.706130
59	Taman Melati	0.00	1	3.223570	101.723990
58	Taman Len Seng	0.00	1	3.069080	101.742870
57	Taman Ibukota	0.00	1	3.212160	101.715400
42	Putrajaya	0.00	1	3.111867	101.713268
53	Taman Bukit Maluri	0.00	1	3.200660	101.633370
54	Taman Cheras Hartamas	0.00	1	3.082630	101.746710
55	Taman Connaught	0.00	1	3.082690	101.736890
0	Alam Damai	0.00	1	3.057690	101.743880
34	Kuchai Lama	0.00	1	3.090740	101.677330
14	Bukit Kiara	0.00	1	3.143480	101.644330
1	Ampang, Kuala Lumpur	0.00	1	3.158093	101.748703
2	Bandar Menjalara	0.00	1	3.190350	101.625450
3	Bandar Sri Permaisuri	0.00	1	3.103910	101.712260
4	Bandar Tasik Selatan	0.01	1	3.072620	101.714710
5	Bandar Tun Razak	0.00	1	3.082800	101.722810
9	Batu 11 Cheras	0.00	1	3.098980	101.734990
10	Batu, Kuala Lumpur	0.00	1	3.098980	101.734990
13	Bukit Jalil	0.00	1	3.057800	101.689650
40	Pantai Dalam	0.01	1	3.095160	101.668120
16	Bukit Petaling	0.01	1	3.129290	101.698920
18	Cheras, Kuala Lumpur	0.00	1	3.061870	101.746750
69	Wangsa Maju	0.01	1	3.203910	101.737190
25	Federal Hill, Kuala Lumpur	0.01	1	3.136370	101.685640
26	Happy Garden	0.01	1	3.201630	101.721070
29	Jinjang	0.00	1	3.209500	101.658740
32	Kampung Datuk Keramat	0.01	1	3.166400	101.730460
33	Kepong	0.00	1	3.217500	101.637630
68	Titiwangsa	0.01	1	3.180670	101.703220
36	Maluri	0.01	1	3.147890	101.694050
38	Miharja	0.01	1	3.147890	101.694050
24	Desa Petaling	0.00	1	3.083310	101.704380

<h2>Cluster 2</h2>

In [ ]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

In [ ]:
Neighborhood	Shopping Mall	Cluster Labels	Latitude	Longitude
11	Brickfields	0.05	2	3.129160	101.684060
30	KL Eco City	0.05	2	3.117110	101.673490
7	Bangsar Park	0.04	2	3.134780	101.672620
6	Bangsar	0.05	2	3.129200	101.678440
35	Lembah Pantai	0.05	2	3.121216	101.663897
66	Taman U-Thant	0.05	2	3.157650	101.725020
41	Pudu, Kuala Lumpur	0.05	2	3.133540	101.713070

<h2>Observations:</h2>

Most of the shopping malls are concentrated in the central area of Kuala Lumpur city, with the highest number in cluster 2 and moderate number in cluster 0.
On the other hand, cluster 1 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 1 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 0 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2 which already have high concentration of shopping malls and suffering from intense competition.